# ParProx

This package aims to perform penalized logistic regression and Cox proportional hazard regression using the proximal gradient method. It supports $\ell_1$ penalty (lasso) and the group lasso penalty. The groups may overlap, as postulated by the latent group lasso ([Obozinski,Jacob, and Vert, 2011](https://arxiv.org/abs/1110.0413)). This package also supports computation on a single Nvidia GPU via the package [CUDA.jl](https://juliagpu.gitlab.io/CUDA.jl/). 

## Installation

### Julia

Download Julia from [https://julialang.org/downloads/](https://julialang.org/downloads/) according to your OS and hardware, and follow platform-specific instruction [here](https://julialang.org/downloads/platform/). 

You can also run Julia from the Jupyter Notebook after installing the package [IJulia](https://github.com/JuliaLang/IJulia.jl):

```julia
using Pkg
pkg"add IJulia"
```

### The Package

The `ParProx` package is installed by using the command:

```julia
using Pkg
pkg"add https://github.com/kose-y/ParProx.jl"
```

To install the other packages used in this tutoral, use the command:
```julia
pkg"add Printf Statistics CSV Mmap CodecZlib ROCAnalysis DelimitedFiles"
```

For optional CUDA support, the following is also needed:
```julia
pkg"add CUDA Adapt"
```

It requires a CUDA-capable GPU with compute capability 5.0 (Maxwell) or higher, and an accompanying Nvidia driver with support for CUDA 10.1 or newer. CUDA is automatically downloaded before the first use, unless configured otherwise through an environment variable. See [this page](https://juliagpu.gitlab.io/CUDA.jl/installation/overview/) for further information. 

The following shows the version of Julia and system information.

In [1]:
versioninfo()

Julia Version 1.5.2
Commit 539f3ce943 (2020-09-23 23:17 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) Silver 4114 CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake-avx512)


## Example 1: Cox regression with nonoverlapping groups

In this example, we perform the Cox regression on somatic mutation count data of 
Glioblastoma patients. We use count of mutations in protein units as the covariates, and the group lasso penalty is applied, where the variables are grouped by the gene where the unit belongs to.

### Loading packages

In [2]:
using ParProx, Printf, Statistics # load the packages
using CSV, DataFrames, CodecZlib, Mmap # packages for data reading. GZip is used to read the gzipped text file.

### Reading data files

The file "somatic_sum_table_GBM.tsv.gz" contains the information the mutations and the count of mutations. Columns 1-9 contains the information of the mutations, and the tenth column and later contains the mutation count for each subject.

In [3]:
somatic = DataFrame!(CSV.File(transcode(GzipDecompressor, Mmap.mmap(
            ParProx.datadir("somatic_sum_table_GBM.tsv.gz")))));

For regular tab- or comma-delimited files, `DataFrame!(CSV.File("test.txt"))` should be enough. 

We print first six rows and five columns of the dataframe:

In [4]:
first(somatic[!, 1:5], 6)

,uniprot_accession,start_position,end_position,center_position,unit_name
,String,String,String,String,String
1,Q9H2S6,NA,NA,NA,NCU
2,O60762,28,199,113,Glycos_transf_2
3,Q8IZE3,26,245,135,Pkinase
4,Q8IZE3,NA,NA,NA,LU
5,Q8IZE3,NA,NA,NA,NCU
6,Q9NSG2,176,728,452,DUF4487


and next four columns are:

In [5]:
first(somatic[!, 6:9], 6)

,gene_name,gene_id,unit_label,sum
,String?,String,String,Int64
1,TNMD,ENSG00000000005,NCU,1
2,DPM1,ENSG00000000419,PIU,1
3,SCYL3,ENSG00000000457,PIU,1
4,SCYL3,ENSG00000000457,LU,2
5,SCYL3,ENSG00000000457,NCU,1
6,C1orf112,ENSG00000000460,PIU,1


The actual count data starts at column 10. 

In [6]:
first(somatic[!, 10:15], 6)

,TCGA-02-0003,TCGA-02-0033,TCGA-02-0047,TCGA-02-0055,TCGA-02-2466,TCGA-02-2470
,Int64,Int64,Int64,Int64,Int64,Int64
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
5,0,0,0,0,0,0
6,0,0,0,0,0,0


The file "primary_TCGA_CDR_GBM.tsv" contains the clinical information of the subjects.

In [7]:
# read data
cdr = CSV.read("primary_TCGA_CDR_GBM.tsv", DataFrame);

In [8]:
first(cdr, 6)

,V1,bcr_patient_barcode,type,age_at_initial_pathologic_diagnosis,gender
,Int64,String,String,Int64,String
1,2647,TCGA-02-0003,GBM,50,MALE
2,2664,TCGA-02-0033,GBM,54,MALE
3,2671,TCGA-02-0047,GBM,78,MALE
4,2676,TCGA-02-0055,GBM,62,FEMALE
5,2734,TCGA-02-2466,GBM,61,MALE
6,2735,TCGA-02-2470,GBM,57,MALE


We take the survival outcome and survival time from the clinical data:

In [9]:
survival_event = convert(Vector, cdr[!, :OS])
survival_time = convert(Vector, cdr[!, Symbol("OS.time")]);

We will also add age at inital pathologic diagnosis and gender as unpenalized covariates. We will code males to 1 and females to 0. 

In [10]:
# age at initial pathological diagnosis
age_at_diagnosis = cdr[!, :age_at_initial_pathologic_diagnosis]

# gender: 1: male, 0: not male
gender = map(x -> x == "MALE", cdr[!, :gender]) 

# we will adjust for these two covariates by adding them as nonpenalized variables. 
clinical = hcat(age_at_diagnosis, gender);

We take the predictors in a single matrix:

In [11]:
somatic_predictors = hcat(convert(Array{Float64}, transpose(convert(Array{Float64}, somatic[:, 10:end]))), clinical)

390×25309 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  50.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  54.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  78.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  62.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  61.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  57.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  43.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  53.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  64.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  81.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  84.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  67.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  63.0  0.0
 ⋮        

Unpenalized variables must be placed at the last columns of the matrix.

we normalize the input variables:

In [12]:
normalize(x) = (x .- mean(x; dims=1))./ std(x; dims=1)
somatic_predictors = normalize(somatic_predictors)

390×25309 Array{Float64,2}:
 -0.050637  -0.050637  -0.050637  …  -0.0879321  -0.748927    0.764111
 -0.050637  -0.050637  -0.050637     -0.0879321  -0.450504    0.764111
 -0.050637  -0.050637  -0.050637     -0.0879321   1.34003     0.764111
 -0.050637  -0.050637  -0.050637     -0.0879321   0.146342   -1.30536
 -0.050637  -0.050637  -0.050637     -0.0879321   0.0717363   0.764111
 -0.050637  -0.050637  -0.050637  …  -0.0879321  -0.226687    0.764111
 -0.050637  -0.050637  -0.050637     -0.0879321  -1.27117     0.764111
 -0.050637  -0.050637  -0.050637     -0.0879321  -0.52511     0.764111
 -0.050637  -0.050637  -0.050637     -0.0879321   0.295553    0.764111
 -0.050637  -0.050637  -0.050637     -0.0879321   1.56385    -1.30536
 -0.050637  -0.050637  -0.050637  …  -0.0879321   1.78767    -1.30536
 -0.050637  -0.050637  -0.050637     -0.0879321   0.519371    0.764111
 -0.050637  -0.050637  -0.050637     -0.0879321   0.220948   -1.30536
  ⋮                               ⋱                  

We are considering 390 subjects and 25309 variables (25307 somatic mutations, age at diagnosis, and gender). 

We check if the order of clinical data and the mutation data matches.

In [13]:
all(cdr[!, :bcr_patient_barcode] .== names(somatic)[10:end])

true

The following code creates a vector of String to be used as the name of each variable. We will use the first eight columns of `somatic` data frame to make a variable name.

In [14]:
somatic_variable_names = map(x -> "$(x[1])\t$(x[2])\t$(x[3])\t" * 
    "$(x[4])\t$(x[5])\t$(x[6])\t$(x[7])\t$(x[8])", eachrow(somatic))
somatic_variable_names = vcat(somatic_variable_names, map(x -> "$x\t\t\t\t\t\t\t", 
        ["age_at_diagnosis", "gender"]));

### Group definition

We figure out unique group names (`somatic_groups`), size of each group (`somatic_group_sizes`), and which group each variable belongs to (`somatic_grpidx`).

In [15]:
somatic_groups = unique(somatic[!, :gene_id])
somatic_group_sizes = map(somatic_groups) do x
    count(y -> y == x, somatic[!, :gene_id])
end
somatic_grpidx = map(somatic[!, :gene_id]) do x 
    findfirst(y -> y .== x, somatic_groups)
end;

`somatic_grpidx` gives the group index each variable belongs. Note that Julia uses 1-based indexing.

In [16]:
somatic_grpidx

25307-element Array{Int64,1}:
     1
     2
     3
     3
     3
     4
     5
     5
     5
     5
     5
     5
     5
     ⋮
 14247
 14248
 14249
 14250
 14251
 14252
 14253
 14253
 14253
 14253
 14253
 14254

### Cross validation

For the cross validation, we create a `COXUpdate` object specifying the update rules: maximum number of iterations, steps between intermediate evaluation, relative tolerance for stopping, and whether to print the intermediate output.

For cross validation in Cox regression, "C-index" is used as the metric (the higher, the better). Stratified k-fold cross validation is implemented (k=5 here), i.e., we ensure that the proportion of event and censored subject as equal as possible across the folds.  

In [17]:
using Random
Random.seed!(222);

We perform 5-fold corss validation using CPU only:

In [18]:
T = Float64
A = Array
U = ParProx.COXUpdate(; maxiter=10000, step=10, tol=5e-4, verbose=true)
lambdas = 10 .^ (range(-5, stop=-7, length=21)) # 21 values equally log-spaced in 10^-5 .. 10^-7
penalties = [GroupNormL2(l, somatic_grpidx) for l in lambdas]

score = ParProx.cross_validate(U, somatic_predictors, 
    survival_event, survival_time, penalties, 5; T=Float64, A=Array)

10	(-1164.4117036781122, Inf, 2.0)
20	(-1164.41032112774, 1.186320686511066e-6, 2.0)
  2.353805 seconds (4.69 M allocations: 252.146 MiB)
10	(-1164.4014882632753, Inf, 14.0)
20	(-1164.39274079207, 7.506028567890206e-6, 14.0)
  0.139054 seconds (1.20 k allocations: 8.068 MiB)
10	(-1164.36733344495, Inf, 54.0)
20	(-1164.34234612163, 2.1442045252264093e-5, 54.0)
  0.138735 seconds (1.08 k allocations: 8.058 MiB)
10	(-1164.2684994741905, Inf, 207.0)
20	(-1164.19693761421, 6.141610715796551e-5, 207.0)
  0.292500 seconds (1.20 k allocations: 8.068 MiB, 52.42% gc time)
10	(-1164.033352342487, Inf, 606.0)
20	(-1163.878455788435, 0.00013297228846683584, 582.0)
  0.132632 seconds (1.08 k allocations: 8.058 MiB)
10	(-1163.5179470390815, Inf, 3648.0)
20	(-1163.252833514157, 0.0002277112988631167, 3461.0)
  0.131808 seconds (1.08 k allocations: 8.058 MiB)
10	(-1162.6646106284334, Inf, 6361.0)
20	(-1162.219399285613, 0.00038274064470879905, 5945.0)
  0.132318 seconds (1.08 k allocations: 8.058 MiB)


70	(-1127.4543587752776, 0.0006021327004472719, 10374.0)
80	(-1126.7915561850416, 0.0005876995501527063, 10325.0)
90	(-1126.1446196925776, 0.0005739605026375526, 10290.0)
100	(-1125.512763646439, 0.0005608955943769029, 10251.0)
110	(-1124.8946331664604, 0.0005490127244324724, 10145.0)
120	(-1124.2899496965254, 0.0005373579228163374, 10117.0)
130	(-1123.6981365554423, 0.0005261973162822256, 10071.0)
140	(-1123.1189081400107, 0.0005152732608954569, 10033.0)
150	(-1122.551695767168, 0.0005048386958781066, 10002.0)
160	(-1121.9959702563121, 0.0004948597551325121, 9976.0)
  1.153135 seconds (9.02 k allocations: 64.495 MiB)
10	(-1121.2559430544381, Inf, 12324.0)
20	(-1120.5353519381567, 0.0006425041484748012, 12293.0)
30	(-1119.8328978877075, 0.0006267250468584852, 12257.0)
40	(-1119.147501018592, 0.0006118809071950223, 12213.0)
50	(-1118.478247140382, 0.0005978266035266739, 12189.0)
60	(-1117.8243054816614, 0.0005844900361179966, 12178.0)
70	(-1117.1849098290002, 0.0005718156693413603, 1214

120	(-1129.0738293851748, 0.0004900540768950248, 11728.0)
  0.869765 seconds (6.86 k allocations: 48.379 MiB)
10	(-1128.3879170989765, Inf, 13821.0)
20	(-1127.7220722074462, 0.0005899104021489063, 13800.0)
30	(-1127.0750137685577, 0.0005735952228273194, 13782.0)
40	(-1126.4456825686532, 0.0005581920349996157, 13767.0)
50	(-1125.8331251968218, 0.0005436096597926845, 13741.0)
60	(-1125.2365088699332, 0.00052974337289707, 13718.0)
70	(-1124.6549175046862, 0.0005166693239667997, 13630.0)
80	(-1124.0876259898657, 0.0005042198507172665, 13628.0)
90	(-1123.5338399095062, 0.0004924583509234823, 13532.0)
  0.597868 seconds (5.05 k allocations: 36.277 MiB)
10	(-1122.8896028973804, Inf, 15412.0)
20	(-1122.2637549786816, 0.0005571691563311305, 15381.0)
30	(-1121.6551892140587, 0.0005420771849359391, 15409.0)
40	(-1121.0629401728904, 0.0005278215864406504, 15389.0)
50	(-1120.4860546008128, 0.000514393888101365, 15317.0)
60	(-1119.9237059694624, 0.0005016832353135907, 15309.0)
70	(-1119.375234285235

10	(-1107.7470388638985, Inf, 20785.0)
20	(-1107.1680677357833, 0.0005224578698591788, 20774.0)
30	(-1106.6024277017655, 0.0005106886910599709, 20779.0)
40	(-1106.0496923000148, 0.0004992868934385925, 20809.0)
  0.251214 seconds (2.29 k allocations: 16.126 MiB)
10	(-1105.4660760332056, Inf, 22568.0)
20	(-1104.8965822053344, 0.0005149611971270356, 22583.0)
30	(-1104.340842181715, 0.0005027770642424179, 22592.0)
40	(-1103.798418614336, 0.0004909706225497378, 22590.0)
  0.258538 seconds (2.16 k allocations: 16.117 MiB)
10	(-1103.2373408494877, Inf, 23020.0)
20	(-1102.6902172866985, 0.0004957220370533697, 23002.0)
  0.128812 seconds (1.20 k allocations: 8.068 MiB)
10	(-1102.1329143072737, Inf, 23446.0)
20	(-1101.590036028561, 0.0004923663927421373, 23449.0)
  0.129028 seconds (1.08 k allocations: 8.059 MiB)
10	(-1101.0432145882417, Inf, 23648.0)
20	(-1100.5109731479674, 0.0004831921363008728, 23660.0)
  0.133088 seconds (1.08 k allocations: 8.059 MiB)
10	(-1099.9793348832789, Inf, 23841.0)

21×5 Array{Float64,2}:
 0.606167  0.532485  0.543738  0.471242  0.438621
 0.605727  0.53294   0.553407  0.472209  0.437701
 0.607048  0.534303  0.554788  0.476559  0.43908
 0.611894  0.537483  0.540055  0.479942  0.44046
 0.618062  0.5393    0.524401  0.481392  0.44
 0.628194  0.544752  0.501381  0.476559  0.44046
 0.637445  0.535666  0.489411  0.468342  0.44092
 0.623348  0.504316  0.474678  0.470275  0.44046
 0.611013  0.489777  0.470994  0.490575  0.43954
 0.606608  0.488869  0.480203  0.498792  0.43954
 0.604405  0.489777  0.482044  0.499275  0.43908
 0.606167  0.491595  0.488029  0.489609  0.438621
 0.606167  0.492049  0.484346  0.487675  0.43954
 0.607489  0.495229  0.486648  0.484775  0.43908
 0.607489  0.497047  0.484346  0.485742  0.438621
 0.60837   0.492049  0.484807  0.486225  0.438621
 0.605727  0.495684  0.484346  0.487192  0.438621
 0.602203  0.495229  0.483886  0.483325  0.43908
 0.595154  0.491595  0.484807  0.483325  0.43954
 0.59207   0.494321  0.486188  0.487192  0.

The number of unpenalized variables are automatically detected from the penalty structure. 

Simple lasso penalty could be used by replacing line 5 above with 
```julia
penalties = [NormL1(l; unpen=2) for l in lambdas]
```
For lasso penalty, the user should provide the number of unpenalized variables at the last columns as a keyword argument.

The resulting c-index for each fold is listed as:

In [19]:
score 

21×5 Array{Float64,2}:
 0.606167  0.532485  0.543738  0.471242  0.438621
 0.605727  0.53294   0.553407  0.472209  0.437701
 0.607048  0.534303  0.554788  0.476559  0.43908
 0.611894  0.537483  0.540055  0.479942  0.44046
 0.618062  0.5393    0.524401  0.481392  0.44
 0.628194  0.544752  0.501381  0.476559  0.44046
 0.637445  0.535666  0.489411  0.468342  0.44092
 0.623348  0.504316  0.474678  0.470275  0.44046
 0.611013  0.489777  0.470994  0.490575  0.43954
 0.606608  0.488869  0.480203  0.498792  0.43954
 0.604405  0.489777  0.482044  0.499275  0.43908
 0.606167  0.491595  0.488029  0.489609  0.438621
 0.606167  0.492049  0.484346  0.487675  0.43954
 0.607489  0.495229  0.486648  0.484775  0.43908
 0.607489  0.497047  0.484346  0.485742  0.438621
 0.60837   0.492049  0.484807  0.486225  0.438621
 0.605727  0.495684  0.484346  0.487192  0.438621
 0.602203  0.495229  0.483886  0.483325  0.43908
 0.595154  0.491595  0.484807  0.483325  0.43954
 0.59207   0.494321  0.486188  0.487192  0.

We select the penalty parameter that maximizes the mean c-index as following:

In [20]:
lambda_idx = argmax(mean(score; dims=2)[:])
lambda = lambdas[lambda_idx]

6.30957344480193e-6

Now we fit the model with the selected value of lambda. 

In [21]:
p = GroupNormL2(lambda, somatic_grpidx)
U = ParProx.COXUpdate(; maxiter=20000, step=20, tol=1e-6, verbose=true)
V = ParProx.COXVariables{Float64, Array}(somatic_predictors, survival_event, survival_time, p; eval_obj=true)
@time ParProx.fit!(U, V)

20	(-1528.605820155985, Inf, 79.0)
40	(-1528.5295022269977, 4.989634320627215e-5, 79.0)
60	(-1528.455726595402, 4.823652644064077e-5, 79.0)
80	(-1528.3843611671928, 4.6662846842996865e-5, 75.0)
100	(-1528.3152631325722, 4.5182335053009895e-5, 74.0)
120	(-1528.2483202337246, 4.377503506912277e-5, 74.0)
140	(-1528.1833903085083, 4.2460522150484454e-5, 72.0)
160	(-1528.120415790859, 4.118349150203354e-5, 72.0)
180	(-1528.0593068836956, 3.9965033984103056e-5, 72.0)
200	(-1527.9999791197333, 3.880167741824099e-5, 72.0)
220	(-1527.9422234428102, 3.7774924413446894e-5, 71.0)
240	(-1527.886034726734, 3.675140906522305e-5, 71.0)
260	(-1527.8313650430123, 3.575913274128511e-5, 70.0)
280	(-1527.778118113276, 3.4829730426855245e-5, 70.0)
300	(-1527.7262788413734, 3.391010713951869e-5, 70.0)
320	(-1527.6757734204632, 3.3038674248892644e-5, 68.0)
340	(-1527.6264722643148, 3.225193141880002e-5, 68.0)
360	(-1527.5784068330806, 3.144453108801795e-5, 68.0)
380	(-1527.5314970130335, 3.068946903537459e-5,

3040	(-1524.541101056502, 7.625416795423652e-6, 38.0)
3060	(-1524.5295531195773, 7.569789061809281e-6, 38.0)
3080	(-1524.5180891521186, 7.514802702233848e-6, 38.0)
3100	(-1524.5067081891293, 7.4604476848144395e-6, 38.0)
3120	(-1524.4954092806652, 7.406714169968683e-6, 38.0)
3140	(-1524.4841914915382, 7.353592511478602e-6, 38.0)
3160	(-1524.4730539010336, 7.301073248163812e-6, 38.0)
3180	(-1524.4619956026293, 7.2491471018141e-6, 38.0)
3200	(-1524.4510157037232, 7.197804972438535e-6, 38.0)
3220	(-1524.440113325369, 7.1470379327686675e-6, 38.0)
3240	(-1524.4292876020152, 7.096837226040329e-6, 38.0)
3260	(-1524.418537681252, 7.047194260197571e-6, 38.0)
3280	(-1524.407862723561, 6.998100607611158e-6, 38.0)
3300	(-1524.3972619020778, 6.949547995107806e-6, 38.0)
3320	(-1524.3867344023492, 6.90152830830846e-6, 38.0)
3340	(-1524.3762794221075, 6.854033580314978e-6, 38.0)
3360	(-1524.365896171041, 6.807055993960672e-6, 38.0)
3380	(-1524.3555838705724, 6.760587877054682e-6, 38.0)
3400	(-1524.3453

6040	(-1523.3835559475103, 3.3589650676325265e-6, 41.0)
6060	(-1523.3784584517123, 3.3439830967119182e-6, 41.0)
6080	(-1523.3733836808842, 3.329086483920131e-6, 41.0)
6100	(-1523.3683315056496, 3.3142745950420285e-6, 41.0)
6120	(-1523.363301797593, 3.2995468013583015e-6, 41.0)
6140	(-1523.3582944292525, 3.2849024791983043e-6, 41.0)
6160	(-1523.3533092741102, 3.2703410108353297e-6, 41.0)
6180	(-1523.3483462065872, 3.2558617821003443e-6, 41.0)
6200	(-1523.3434051020329, 3.2414641856638276e-6, 41.0)
6220	(-1523.3384858367192, 3.227147618052852e-6, 41.0)
6240	(-1523.3335882878318, 3.2129114814413103e-6, 41.0)
6260	(-1523.3287123334628, 3.1987551829044025e-6, 41.0)
6280	(-1523.3238578526036, 3.1846781339714256e-6, 41.0)
6300	(-1523.319024725138, 3.1706797509243793e-6, 41.0)
6320	(-1523.3142128318316, 3.1567594567373844e-6, 41.0)
6340	(-1523.3094220543296, 3.142916676005369e-6, 41.0)
6360	(-1523.3046522751456, 3.1291508406125827e-6, 41.0)
6380	(-1523.2999033776566, 3.1154613855562547e-6, 41.

9020	(-1522.8199002837432, 1.8073370336715487e-6, 41.0)
9040	(-1522.8171570934594, 1.8002096058829354e-6, 41.0)
9060	(-1522.814424722491, 1.7931126810178935e-6, 41.0)
9080	(-1522.8117031245874, 1.7860460697951963e-6, 41.0)
9100	(-1522.8089922537838, 1.7790095854664214e-6, 41.0)
9120	(-1522.8062920644018, 1.772003040085644e-6, 41.0)
9140	(-1522.8036025110428, 1.7650262504780453e-6, 41.0)
9160	(-1522.8009235485888, 1.7580790328682483e-6, 41.0)
9180	(-1522.7982551322002, 1.7511612049693535e-6, 41.0)
9200	(-1522.7955972173118, 1.7442725870274936e-6, 41.0)
9220	(-1522.7929497596317, 1.7374129998820786e-6, 41.0)
9240	(-1522.7903127151396, 1.730582265861127e-6, 41.0)
9260	(-1522.787686040084, 1.7237802087813084e-6, 41.0)
9280	(-1522.78506969098, 1.7170066540972005e-6, 41.0)
9300	(-1522.7824636246075, 1.7102614281552482e-6, 41.0)
9320	(-1522.7798677980084, 1.703544359685971e-6, 41.0)
9340	(-1522.7772821684873, 1.6968552762227934e-6, 41.0)
9360	(-1522.7747066936047, 1.6901940105184174e-6, 41.0)

11940	(-1522.504941339718, 1.0566022533708992e-6, 39.0)
11960	(-1522.5033383836337, 1.0521513434940807e-6, 39.0)
11980	(-1522.5017421853142, 1.047716766760024e-6, 39.0)
12000	(-1522.5001527199786, 1.0432984419579184e-6, 39.0)
12020	(-1522.498569962968, 1.0388962889205217e-6, 39.0)
12040	(-1522.4969938897434, 1.0345102294196442e-6, 39.0)
12060	(-1522.4954244758876, 1.0301401832858002e-6, 39.0)
12080	(-1522.4938616971024, 1.025786072736311e-6, 39.0)
12100	(-1522.492305529208, 1.0214478201366488e-6, 39.0)
12120	(-1522.4907559481449, 1.0171253465079921e-6, 39.0)
12140	(-1522.4892129299687, 1.012818576601572e-6, 39.0)
12160	(-1522.487676450854, 1.0085274324811386e-6, 39.0)
12180	(-1522.4861464870896, 1.004251839045048e-6, 39.0)
12200	(-1522.4846230150818, 9.999917195489075e-7, 39.0)
 87.642930 seconds (599.45 k allocations: 4.738 GiB, 0.57% gc time)


We obtain the indexes of variables with nonzero coefficients:

In [22]:
nonzero_idxs = (1:length(V.β))[V.β .!= 0];

The selected variables, along with the coefficients can be printed as below:

In [23]:
for (variable_name, value) in zip(somatic_variable_names[nonzero_idxs], V.β[nonzero_idxs])
    println(variable_name, "\t", value)
end

Q92782	39	110	74	Requiem_N	DPF1	ENSG00000011332	PIU	-0.0019080081184242276
Q13492	21	284	152	ANTH	PICALM	ENSG00000073921	PIU	-0.005111512997849868
Q08116	85	199	142	RGS	RGS1	ENSG00000090104	PIU	-0.00048398156368121485
P36897	205	492	348	Pkinase	TGFBR1	ENSG00000106799	PIU	-0.00048398156368121485
O76050	65	217	141	Neuralized	NEURL1	ENSG00000107954	PIU	-0.005316663166758178
P35680	NA	NA	NA	NCU	HNF1B	ENSG00000108753	NCU	-0.0004000403144672045
Q13526	NA	NA	NA	NCU	PIN1	ENSG00000127445	NCU	-0.004373984418609586
Q93097	74	380	227	wnt	WNT2B	ENSG00000134245	PIU	-0.005316663166758178
Q5VWQ8	646	1159	902	DUF3498	DAB2IP	ENSG00000136848	PIU	-0.0004000403144672045
O75874	9	401	205	Iso_dh	IDH1	ENSG00000138413	PIU	-0.012790481289534012
O75874	130	140	135	py	IDH1	ENSG00000138413	PIU	-0.012790481289534012
Q7L5Y6	142	547	344	Det1	DET1	ENSG00000140543	PIU	-0.0026803841864418343
Q7L5Y6	NA	NA	NA	LU	DET1	ENSG00000140543	LU	-0.00249743745707543
Q8TD46	NA	NA	NA	NCU	CD200R1	ENSG00000163606	NCU	-0.000483981563681

### (optional) Analysis using GPU

For using GPU, as stated before, the package CUDA.jl is required. 

Cross validation on GPU is performed by the following code. 

In [24]:
using CUDA, Adapt

The following identifies the GPU we are running the code on.

In [25]:
CUDA.device()

CuDevice(0): TITAN V

It takes only a minor modification to run the cross validation on GPU. 

In [26]:
Random.seed!(222)
T = Float64
A = CuArray
U = ParProx.COXUpdate(; maxiter=10000, step=10, tol=5e-4, verbose=true)
lambdas = 10 .^ (range(-5, stop=-7, length=21)) # 21 values equally log-spaced in 10^-5 .. 10^-7
penalties = [GroupNormL2{T,A}(l, somatic_grpidx) for l in lambdas]

score = ParProx.cross_validate(U, somatic_predictors, 
    survival_event, survival_time, penalties, 5; T=T, A=A)

10	(-1164.4117036781122, Inf, 2.0)
20	(-1164.4103211277397, 1.186320686706168e-6, 2.0)
 12.524966 seconds (23.35 M allocations: 1.183 GiB, 1.90% gc time)
10	(-1164.4014882632753, Inf, 14.0)
20	(-1164.39274079207, 7.506028567890206e-6, 14.0)
  0.023262 seconds (12.77 k allocations: 20.133 MiB)
10	(-1164.3673334449495, Inf, 54.0)
20	(-1164.34234612163, 2.1442045251873865e-5, 54.0)
  0.031365 seconds (12.96 k allocations: 20.126 MiB)
10	(-1164.2684994741905, Inf, 207.0)
20	(-1164.1969376142097, 6.141610715816066e-5, 207.0)
  0.031352 seconds (12.98 k allocations: 20.130 MiB)
10	(-1164.033352342487, Inf, 606.0)
20	(-1163.878455788435, 0.00013297228846683584, 582.0)
  0.036319 seconds (13.10 k allocations: 20.138 MiB)
10	(-1163.5179470390815, Inf, 3648.0)
20	(-1163.252833514157, 0.0002277112988631167, 3461.0)
  0.034382 seconds (12.97 k allocations: 20.126 MiB)
10	(-1162.6646106284331, Inf, 6361.0)
20	(-1162.2193992856128, 0.0003827406447087991, 5945.0)
  0.040597 seconds (12.98 k allocatio

30	(-1130.2826722616473, 0.0006692613464204281, 10580.0)
40	(-1129.5468732322374, 0.0006508346065353895, 10508.0)
50	(-1128.8309839517333, 0.0006336251091293547, 10458.0)
60	(-1128.1338380456589, 0.0006174165387524418, 10403.0)
70	(-1127.4543587752778, 0.0006021327004474733, 10374.0)
80	(-1126.7915561850418, 0.0005876995501527062, 10325.0)
90	(-1126.144619692578, 0.0005739605026373506, 10290.0)
100	(-1125.512763646439, 0.0005608955943773065, 10251.0)
110	(-1124.8946331664606, 0.0005490127244322704, 10145.0)
120	(-1124.2899496965258, 0.0005373579228161351, 10117.0)
130	(-1123.6981365554425, 0.0005261973162824276, 10071.0)
140	(-1123.1189081400112, 0.0005152732608952544, 10033.0)
150	(-1122.5516957671684, 0.0005048386958781064, 10002.0)
160	(-1121.9959702563124, 0.0004948597551327146, 9976.0)
  0.139079 seconds (100.44 k allocations: 160.966 MiB)
10	(-1121.2559430544384, Inf, 12324.0)
20	(-1120.535351938157, 0.000642504148474801, 12293.0)
30	(-1119.8328978877078, 0.000626725046858485, 12

100	(-1130.1947509664867, 0.0005135663955427244, 11841.0)
110	(-1129.6276266724574, 0.000501601305903365, 11759.0)
120	(-1129.0738293851748, 0.0004900540768950248, 11728.0)
  0.096708 seconds (75.27 k allocations: 120.718 MiB)
10	(-1128.387917098977, Inf, 13821.0)
20	(-1127.7220722074464, 0.0005899104021491077, 13800.0)
30	(-1127.075013768558, 0.0005735952228273193, 13782.0)
40	(-1126.4456825686534, 0.0005581920349996155, 13767.0)
50	(-1125.833125196822, 0.0005436096597926844, 13741.0)
60	(-1125.2365088699335, 0.0005297433728970699, 13718.0)
70	(-1124.6549175046869, 0.0005166693239663954, 13630.0)
80	(-1124.087625989866, 0.0005042198507176705, 13628.0)
90	(-1123.5338399095067, 0.00049245835092328, 13532.0)
  0.080800 seconds (64.52 k allocations: 90.687 MiB, 7.74% gc time)
10	(-1122.8896028973807, Inf, 15412.0)
20	(-1122.2637549786816, 0.0005571691563313329, 15381.0)
30	(-1121.6551892140592, 0.0005420771849355339, 15409.0)
40	(-1121.0629401728906, 0.000527821586440853, 15389.0)
50	(-11

60	(-1108.8854228044684, 0.0005022734161659926, 18462.0)
70	(-1108.3397500806066, 0.0004918896341920039, 18996.0)
  0.057562 seconds (43.95 k allocations: 70.423 MiB)
10	(-1107.7470388638983, Inf, 20785.0)
20	(-1107.1680677357833, 0.0005224578698589735, 20774.0)
30	(-1106.6024277017655, 0.0005106886910599709, 20779.0)
40	(-1106.0496923000144, 0.0004992868934390035, 20809.0)
  0.033526 seconds (25.05 k allocations: 40.235 MiB)
10	(-1105.4660760332054, Inf, 22568.0)
20	(-1104.8965822053344, 0.0005149611971268299, 22583.0)
30	(-1104.3408421817146, 0.0005027770642428294, 22592.0)
40	(-1103.7984186143358, 0.000490970622549532, 22590.0)
  0.042302 seconds (33.33 k allocations: 40.387 MiB, 14.88% gc time)
10	(-1103.2373408494877, Inf, 23020.0)
20	(-1102.6902172866983, 0.0004957220370535758, 23002.0)
  0.018449 seconds (12.65 k allocations: 20.130 MiB)
10	(-1102.1329143072737, Inf, 23446.0)
20	(-1101.5900360285607, 0.0004923663927423436, 23449.0)
  0.017207 seconds (12.55 k allocations: 20.128

21×5 Array{Float64,2}:
 0.606167  0.53294   0.542818  0.471242  0.438621
 0.606167  0.532485  0.554328  0.471725  0.437701
 0.607048  0.535211  0.555249  0.478009  0.43908
 0.611894  0.537483  0.540976  0.480425  0.44046
 0.618062  0.5393    0.524401  0.481392  0.44
 0.628194  0.544752  0.501381  0.476559  0.44046
 0.637445  0.535666  0.489411  0.468342  0.44092
 0.623348  0.504316  0.474678  0.470275  0.44046
 0.611013  0.489777  0.470994  0.490575  0.43954
 0.606608  0.488869  0.480203  0.498792  0.43954
 0.604405  0.489777  0.482044  0.499275  0.43908
 0.606167  0.491595  0.488029  0.489609  0.438621
 0.606167  0.492049  0.484346  0.487675  0.43954
 0.607489  0.495229  0.486648  0.484775  0.43908
 0.607489  0.497047  0.484346  0.485742  0.438621
 0.60837   0.492049  0.484807  0.486225  0.438621
 0.605727  0.495684  0.484346  0.487192  0.438621
 0.602203  0.495229  0.483886  0.483325  0.43908
 0.595154  0.491595  0.484807  0.483325  0.43954
 0.59207   0.494321  0.486188  0.487192  0.

It can be seen that analysis on GPU is 6-8 times faster except for the very first fit where the functions are compiled. It may be desirable to use `T=Float32` to reduce memory usage of the GPU. 

In [27]:
lambda_idx = argmax(mean(score; dims=2)[:])
lambda = lambdas[lambda_idx]

6.30957344480193e-6

In [28]:
p = GroupNormL2{T,A}(lambda, somatic_grpidx)
U = ParProx.COXUpdate(; maxiter=20000, step=20, tol=1e-6, verbose=true)
V = ParProx.COXVariables{T,A}(adapt(A{T}, somatic_predictors), 
    adapt(A{T}, survival_event), 
    adapt(A{T}, survival_time), p; eval_obj=true)
@time ParProx.fit!(U, V)

20	(-1528.605820155985, Inf, 79.0)
40	(-1528.5295022269977, 4.989634320627215e-5, 79.0)
60	(-1528.455726595402, 4.823652644064077e-5, 79.0)
80	(-1528.3843611671928, 4.6662846842996865e-5, 75.0)
100	(-1528.3152631325722, 4.5182335053009895e-5, 74.0)
120	(-1528.2483202337244, 4.377503506927146e-5, 74.0)
140	(-1528.1833903085083, 4.246052215033576e-5, 72.0)
160	(-1528.1204157908592, 4.118349150188484e-5, 72.0)
180	(-1528.0593068836954, 3.9965033984400466e-5, 72.0)
200	(-1527.9999791197333, 3.880167741809229e-5, 72.0)
220	(-1527.9422234428102, 3.7774924413446894e-5, 71.0)
240	(-1527.8860347267334, 3.67514090655205e-5, 71.0)
260	(-1527.8313650430123, 3.575913274098766e-5, 70.0)
280	(-1527.7781181132762, 3.482973042670651e-5, 70.0)
300	(-1527.7262788413732, 3.391010713981616e-5, 70.0)
320	(-1527.675773420463, 3.303867424889265e-5, 68.0)
340	(-1527.6264722643148, 3.225193141865128e-5, 68.0)
360	(-1527.5784068330804, 3.144453108816671e-5, 68.0)
380	(-1527.5314970130335, 3.068946903522584e-5, 6

3020	(-1524.5527339432358, 7.6816961382687e-6, 38.0)
3040	(-1524.541101056502, 7.625416795274608e-6, 38.0)
3060	(-1524.5295531195775, 7.569789061660234e-6, 38.0)
3080	(-1524.5180891521188, 7.514802702233847e-6, 38.0)
3100	(-1524.5067081891293, 7.460447684963488e-6, 38.0)
3120	(-1524.4954092806652, 7.406714169968683e-6, 38.0)
3140	(-1524.4841914915382, 7.353592511478602e-6, 38.0)
3160	(-1524.4730539010334, 7.301073248312864e-6, 38.0)
3180	(-1524.4619956026295, 7.249147101515994e-6, 38.0)
3200	(-1524.4510157037232, 7.197804972587588e-6, 38.0)
3220	(-1524.440113325369, 7.1470379327686675e-6, 38.0)
3240	(-1524.4292876020152, 7.096837226040329e-6, 38.0)
3260	(-1524.4185376812518, 7.04719426034663e-6, 38.0)
3280	(-1524.407862723561, 6.998100607462101e-6, 38.0)
3300	(-1524.3972619020776, 6.949547995256866e-6, 38.0)
3320	(-1524.3867344023492, 6.901528308159401e-6, 38.0)
3340	(-1524.3762794221075, 6.854033580314978e-6, 38.0)
3360	(-1524.3658961710407, 6.807055994109735e-6, 38.0)
3380	(-1524.355

6040	(-1523.3835559475103, 3.3589650677816843e-6, 41.0)
6060	(-1523.3784584517123, 3.3439830967119182e-6, 41.0)
6080	(-1523.3733836808842, 3.329086483920131e-6, 41.0)
6100	(-1523.3683315056496, 3.3142745950420285e-6, 41.0)
6120	(-1523.363301797593, 3.2995468013583015e-6, 41.0)
6140	(-1523.3582944292523, 3.284902479347465e-6, 41.0)
6160	(-1523.35330927411, 3.27034101083533e-6, 41.0)
6180	(-1523.3483462065872, 3.255861781951183e-6, 41.0)
6200	(-1523.3434051020329, 3.2414641856638276e-6, 41.0)
6220	(-1523.3384858367194, 3.227147617903689e-6, 41.0)
6240	(-1523.3335882878318, 3.2129114815904732e-6, 41.0)
6260	(-1523.3287123334628, 3.1987551829044025e-6, 41.0)
6280	(-1523.323857852604, 3.1846781336730977e-6, 41.0)
6300	(-1523.319024725138, 3.1706797512227073e-6, 41.0)
6320	(-1523.3142128318318, 3.1567594565882194e-6, 41.0)
6340	(-1523.3094220543298, 3.1429166760053684e-6, 41.0)
6360	(-1523.3046522751456, 3.1291508407617486e-6, 41.0)
6380	(-1523.2999033776564, 3.1154613857054214e-6, 41.0)
640

9040	(-1522.8171570934592, 1.800209606032149e-6, 41.0)
9060	(-1522.814424722491, 1.7931126808686802e-6, 41.0)
9080	(-1522.8117031245872, 1.7860460699444103e-6, 41.0)
9100	(-1522.8089922537838, 1.7790095853172072e-6, 41.0)
9120	(-1522.8062920644018, 1.772003040085644e-6, 41.0)
9140	(-1522.8036025110425, 1.76502625062726e-6, 41.0)
9160	(-1522.8009235485886, 1.7580790328682485e-6, 41.0)
9180	(-1522.7982551322002, 1.7511612048201385e-6, 41.0)
9200	(-1522.7955972173115, 1.7442725871767092e-6, 41.0)
9220	(-1522.7929497596317, 1.737412999732863e-6, 41.0)
9240	(-1522.7903127151399, 1.730582265711911e-6, 41.0)
9260	(-1522.787686040084, 1.7237802089305245e-6, 41.0)
9280	(-1522.7850696909798, 1.717006654246417e-6, 41.0)
9300	(-1522.7824636246073, 1.7102614281552487e-6, 41.0)
9320	(-1522.7798677980086, 1.703544359387537e-6, 41.0)
9340	(-1522.7772821684873, 1.6968552763720105e-6, 41.0)
9360	(-1522.7747066936047, 1.6901940105184174e-6, 41.0)
9380	(-1522.7721413311792, 1.6835603932332238e-6, 41.0)
94

11980	(-1522.5017421853145, 1.0477167666107798e-6, 39.0)
12000	(-1522.5001527199786, 1.0432984421071627e-6, 39.0)
12020	(-1522.498569962968, 1.0388962889205217e-6, 39.0)
12040	(-1522.4969938897436, 1.0345102292703995e-6, 39.0)
12060	(-1522.4954244758876, 1.0301401834350449e-6, 39.0)
12080	(-1522.4938616971024, 1.025786072736311e-6, 39.0)
12100	(-1522.4923055292084, 1.0214478198381584e-6, 39.0)
12120	(-1522.490755948145, 1.0171253466572372e-6, 39.0)
12140	(-1522.4892129299687, 1.0128185767508173e-6, 39.0)
12160	(-1522.487676450854, 1.0085274324811386e-6, 39.0)
12180	(-1522.4861464870899, 1.0042518388958022e-6, 39.0)
12200	(-1522.4846230150818, 9.999917196981532e-7, 39.0)
 11.828392 seconds (7.79 M allocations: 11.870 GiB, 3.23% gc time)


In [29]:
nonzero_idxs = (1:length(V.β))[V.β .!= 0];

┌ Warning: Performing scalar operations on GPU arrays: This is very slow, consider disallowing these operations with `allowscalar(false)`
└ @ GPUArrays /home/kose/.julia/packages/GPUArrays/jhRU7/src/host/indexing.jl:43


In [30]:
for (variable_name, value) in zip(somatic_variable_names[nonzero_idxs], V.β[nonzero_idxs])
    println(variable_name, "\t", value)
end

Q92782	39	110	74	Requiem_N	DPF1	ENSG00000011332	PIU	-0.0019080081184242286
Q13492	21	284	152	ANTH	PICALM	ENSG00000073921	PIU	-0.005111512997849875
Q08116	85	199	142	RGS	RGS1	ENSG00000090104	PIU	-0.0004839815636812231
P36897	205	492	348	Pkinase	TGFBR1	ENSG00000106799	PIU	-0.0004839815636812231
O76050	65	217	141	Neuralized	NEURL1	ENSG00000107954	PIU	-0.005316663166758194
P35680	NA	NA	NA	NCU	HNF1B	ENSG00000108753	NCU	-0.00040004031446720623
Q13526	NA	NA	NA	NCU	PIN1	ENSG00000127445	NCU	-0.0043739844186095915
Q93097	74	380	227	wnt	WNT2B	ENSG00000134245	PIU	-0.005316663166758194
Q5VWQ8	646	1159	902	DUF3498	DAB2IP	ENSG00000136848	PIU	-0.00040004031446720623
O75874	9	401	205	Iso_dh	IDH1	ENSG00000138413	PIU	-0.012790481289534065
O75874	130	140	135	py	IDH1	ENSG00000138413	PIU	-0.012790481289534065
Q7L5Y6	142	547	344	Det1	DET1	ENSG00000140543	PIU	-0.0026803841864418724
Q7L5Y6	NA	NA	NA	LU	DET1	ENSG00000140543	LU	-0.002497437457075466
Q8TD46	NA	NA	NA	NCU	CD200R1	ENSG00000163606	NCU	-0.0004839815636

## Example 2: Logistic regression with overlapping groups

In this example, we perform logistic regression with overlapping group lasso penalties. This package provides a specialized interface for overlapping groups. We use mRNA gene expression signature to classify breast cancer patients undergoing chemotherapy with anthracycline and neoadjuvant agents into two classes. Group of variables is defined by the pathway and GO terms. Genes not belonging to any of the pathways is included as a singleton group. 

In latent group lasso model, variables in multiple groups are replicated each time, each replication corresponding to the contribution of a variable as a member of each group. We estimate the corresponding coefficient for each replication, and the sum of these replicated coefficients is the total coefficient for that variable. This is implemented as a lazy multiplication between the original data and a sparse matrix, reducing the memory usage. 


For this analysis, we have four data files.

- `exprdata_set2_cut.txt.gz` contains the gene expression data of the subjects. 
- `sample_set2.txt.gz` contains the clinical information of the subjects.
- `PATHWAYgroup.txt.gz` contains the information on the group structure: membership of each gene in a pathway. 
- `gene_info.txt.gz` matches gene symbols to their functions.

For the preprocessing, we first read the gene expression and clinical data, then parse `PATHWAYgroup.txt.gz` to determine the group structure. In the process, we also attach the functions to each of the genes and pathways. Then, we construct variable names from the group structure and the functions.

### Reading covariates

The following lines read the gene expression data. The first column of the file `exprdata_set2_cut.txt.gz` has the gene names, and the other columns represent the normalized gene expression level of each patient. We extract the gene names for later use.

In [31]:
brca_data = DataFrame!(CSV.File(transcode(GzipDecompressor, Mmap.mmap(
            ParProx.datadir("exprdata_set2_cut.txt.gz")))))
gene_names = brca_data[!, :Gene]

1043-element Array{String,1}:
 "RFC2"
 "PAX8"
 "THRA"
 "EPHB3"
 "CFL1"
 "YY1"
 "ZPR1"
 "RHOA"
 "GUK1"
 "DSP"
 "RAD21"
 "SF3B2"
 "NDRG1"
 ⋮
 "ORAI3"
 "FAM174B"
 "ZNF335"
 "SEH1L"
 "LOC389906"
 "KIF18B"
 "RACGAP1"
 "CASP8AP2"
 "KLK5"
 "OR7E47P"
 "SCAF4"
 "SNHG17"

Now we load the covariates. Be carful when matching the subject information on the gene expression data and clinical data. We gather penalized variables and unpenalized variables separately. 

In [32]:
# patient names in the gene expression file
patient_barcode = map(x -> string(x), names(brca_data)[2:end])

X = collect(transpose(convert(Matrix, brca_data[:, 2:end])))

subject_data = CSV.File(transcode(GzipDecompressor, Mmap.mmap(
            ParProx.datadir("sample_set2.txt.gz")))) |> DataFrame
normalize(x) = (x .- mean(x; dims=1))./ std(x; dims=1)
Age  = normalize(subject_data[!, :Age])
interceptcol = ones(size(X, 1))

X_unpen = [Age interceptcol] 

y = subject_data[!, :PCR_outcome]

# match subject ids
subject_data = filter(x -> x[:GEO_accession] in patient_barcode, subject_data) 
@assert all(subject_data[!, :GEO_accession] .== patient_barcode)

### Building the groups

Now we read the group information. Each row of the file `PATHWAYgroup.txt.gz` lists a gene, a pathway id, function of the pathway and the gene symbol, indicating the membership of the gene in the pathway. We read the data file first:

In [33]:
group_info = DataFrame!(CSV.File(transcode(GzipDecompressor, Mmap.mmap(
            ParProx.datadir("PATHWAYgroup.txt.gz")))))
head(group_info)

,ENSG,Pathwayid,Function,Genesym
,String,String,String,String
1,ENSG00000148584,GO:0006397,mRNA processing,A1CF
2,ENSG00000148584,GO:0010467,gene expression,A1CF
3,ENSG00000148584,GO:0016554,cytidine to uridine editing,A1CF
4,ENSG00000148584,GO:0016556,mRNA modification,A1CF
5,ENSG00000148584,GO:0050821,protein stabilization,A1CF
6,ENSG00000175899,KEGG:04610,Complement and coagulation cascades,A2M


We create variables to store the group information. 

- `gene_names` (defined above): Each element lists the gene symbols in the analysis. Location in this list matches the index of each gene.
- `pathwayids`: A unique list of pathway ids. Location in this list determines the index of each pathway.
- `variable_to_groups`: Each element lists the group each variable is in.
- `group_to_variables`: Each element lists the variable each group includes.
- `genesym_to_variableidx`: Matches gene symbol to variable index.
- `pathwayid_to_groupidx`: Matches pathway id to group index. 

In [34]:
pathwayids = Vector{String}()
variable_to_groups = [Int[] for i in 1:length(gene_names)] 
group_to_variables = Vector{Vector{Int}}()
genesym_to_variableidx = Dict{String, Int}()
pathwayid_to_groupidx = Dict{String, Int}();

These dictionaries map pathwayids and gene symbols to their functions. 

In [35]:
pathwayid_to_pathwayfunction = Dict{String, String}()
genesym_to_genefunction = Dict{String, String}();

We build `genesym_to_variableidx` and `genesym_to_genefunction`: 

In [36]:
for (i, v) in enumerate(gene_names)
    genesym_to_variableidx[v] = i
end

gene_info = CSV.File(transcode(GzipDecompressor, Mmap.mmap(
            ParProx.datadir("gene_info.txt.gz")))) |> DataFrame
for r in eachrow(gene_info)
    genesym_to_genefunction[r[2]] = r[4]
end

`pathwayids`, `variable_to_groups`, `group_to_variables`, `pathwayid_to_groupidx`, and `pathwayid_to_pathwayfunction` are all built by parsing `group_info`.  

In [37]:
num_groups = 0
for r in eachrow(group_info)
    if !(r.Genesym in keys(genesym_to_variableidx)) # skip any gene not in our gene list
        continue
    end
    # get variable index from the gene symbol
    variable_idx = genesym_to_variableidx[r.Genesym]
    
    if !(r.Pathwayid in pathwayids) # if the pathway id is new ... #!haskey(pathwayid_to_genesym, r.Pathwayid) 
        # add a new group
        num_groups += 1
        push!(pathwayids, r.Pathwayid)
        push!(group_to_variables, Int[])
        pathwayid_to_groupidx[r.Pathwayid] = num_groups
        group_idx = num_groups
    else
        group_idx = pathwayid_to_groupidx[r.Pathwayid]
    end
    
    # insert membership information
    push!(variable_to_groups[variable_idx], group_idx)
    push!(group_to_variables[group_idx], variable_idx)
    
    # retrieve pathway information
    if !haskey(pathwayid_to_pathwayfunction, r.Pathwayid)
        pathwayid_to_pathwayfunction[r.Pathwayid] = r.Function
    else
        # if redundant, just check correctness
        @assert pathwayid_to_pathwayfunction[r.Pathwayid] == r.Function
    end
end



We also enter non-pathway genes as singleton groups.

In [38]:
cnt = 0
for (i, v) in enumerate(gene_names)
    if length(variable_to_groups[i]) > 0 # gene is in some of the groups
        continue
    else # gene is not in any of the groups
        num_groups += 1
        cnt += 1
        # update group info
        push!(variable_to_groups[i], num_groups)
        push!(group_to_variables, [i])
        
        # update variable names
        pathwayid_placeholder = 
            if haskey(genesym_to_genefunction, v)
                "Singleton of: $v"
            else # some genes do not appear in the gene function list
                "Singleton of: $v"
            end
        push!(pathwayids, pathwayid_placeholder)
    end
end
cnt

90

90 singleton groups were added!

### Constructing variable names

Here we create a vector of Strings for pathway ids and their functions.

First, we create a vector of variable names. We include age and intercept as nonpenalized variables. 

In [39]:
variable_names = [gene_names; "Age\t\t"; "intercept\t\t"]

1045-element Array{String,1}:
 "RFC2"
 "PAX8"
 "THRA"
 "EPHB3"
 "CFL1"
 "YY1"
 "ZPR1"
 "RHOA"
 "GUK1"
 "DSP"
 "RAD21"
 "SF3B2"
 "NDRG1"
 ⋮
 "ZNF335"
 "SEH1L"
 "LOC389906"
 "KIF18B"
 "RACGAP1"
 "CASP8AP2"
 "KLK5"
 "OR7E47P"
 "SCAF4"
 "SNHG17"
 "Age\t\t"
 "intercept\t\t"

We have 1045 variables for the analysis. 

Now we concatenate pathway names and their functions. 

In [40]:
pathways_with_functions = map(pathwayids) do x
    if haskey(pathwayid_to_pathwayfunction, x)
        "$x: $(pathwayid_to_pathwayfunction[x])"
    else
        x
    end
end

5719-element Array{String,1}:
 "KEGG:00280: Valine, leucine and isoleucine degradation"
 "KEGG:04727: GABAergic synapse"
 "KEGG:01100: Metabolic pathways"
 "KEGG:00250: Alanine, aspartate and glutamate metabolism"
 "KEGG:00640: Propanoate metabolism"
 "KEGG:00410: beta-Alanine metabolism"
 "KEGG:00650: Butanoate metabolism"
 "GO:0001666: response to hypoxia"
 "GO:0007268: synaptic transmission"
 "GO:0007269: neurotransmitter secretion"
 "GO:0007620: copulation"
 "GO:0007626: locomotory behavior"
 "GO:0009449: gamma-aminobutyric acid biosynthetic process"
 ⋮
 "Singleton of: TEX13B"
 "Singleton of: IQCG"
 "Singleton of: LOC100131532"
 "Singleton of: C6orf25"
 "Singleton of: TMEM14B"
 "Singleton of: FAM64A"
 "Singleton of: GPR124"
 "Singleton of: FAM174B"
 "Singleton of: LOC389906"
 "Singleton of: OR7E47P"
 "Singleton of: SCAF4"
 "Singleton of: SNHG17"

We now create replicated variable names: each replicated variable corresponds to a membership of a gene to a group. The name should include pathway information and group information. 

In [41]:
variable_names_replicated = String[]
cnt = 0
for i in 1:length(group_to_variables)
    for v in group_to_variables[i]
        pf = pathways_with_functions[i]
        g = gene_names[v]
        gt = genesym_to_genefunction[g]
        push!(variable_names_replicated, "$pf\t$g\t$gt")
    end
end

Finally, we append nonpenalized variables to the list. 

In [42]:
variable_names_replicated = [variable_names_replicated; "Age\t\t"; "intercept\t\t"] 

21965-element Array{String,1}:
 "KEGG:00280: Valine, leucine and isoleucine degradation\tABAT\t4-aminobutyrate aminotransferase"
 "KEGG:00280: Valine, leucine and isoleucine degradation\tAUH\tAU RNA binding protein/enoyl-CoA hydratase"
 "KEGG:00280: Valine, leucine and isoleucine degradation\tHMGCL\t3-hydroxymethyl-3-methylglutaryl-CoA lyase"
 "KEGG:00280: Valine, leucine and isoleucine degradation\tIVD\tisovaleryl-CoA dehydrogenase"
 "KEGG:00280: Valine, leucine and isoleucine degradation\tMCCC1\tmethylcrotonoyl-CoA carboxylase 1 (alpha)"
 "KEGG:00280: Valine, leucine and isoleucine degradation\tMCCC2\tmethylcrotonoyl-CoA carboxylase 2 (beta)"
 "KEGG:00280: Valine, leucine and isoleucine degradation\tMUT\tmethylmalonyl CoA mutase"
 "KEGG:04727: GABAergic synapse\tABAT\t4-aminobutyrate aminotransferase"
 "KEGG:04727: GABAergic synapse\tGABRA4\tgamma-aminobutyric acid (GABA) A receptor, alpha 4"
 "KEGG:04727: GABAergic synapse\tPRKCA\tprotein kinase C, alpha"
 "KEGG:04727: GABAergic syn

We have 21965 variables on the replicated space.

### Cross validation

Now we perform the 5-fold cross validation. The default criteria for cross validation is accuracy. It can be changed to area under the ROC curve (ROC) by adding a keyword parameter `criteria=auc`.

In [43]:
lambdas = 10 .^ (range(-6, stop=-9, length=31))

31-element Array{Float64,1}:
 1.0e-6
 7.943282347242822e-7
 6.30957344480193e-7
 5.011872336272725e-7
 3.981071705534969e-7
 3.162277660168379e-7
 2.5118864315095823e-7
 1.9952623149688787e-7
 1.584893192461114e-7
 1.2589254117941662e-7
 1.0e-7
 7.943282347242822e-8
 6.30957344480193e-8
 ⋮
 1.2589254117941661e-8
 1.0e-8
 7.943282347242822e-9
 6.309573444801943e-9
 5.011872336272715e-9
 3.981071705534969e-9
 3.1622776601683795e-9
 2.511886431509582e-9
 1.9952623149688828e-9
 1.584893192461111e-9
 1.2589254117941663e-9
 1.0e-9

For overlapping groups, `group_to_variables` is used as an input, along with `lambdas`, rather than the pre-constructed penalties. 

In [44]:
using Random
Random.seed!(222)
U = ParProx.LogisticUpdate(; maxiter=30000, step=20, tol=5e-4, verbose=true)
@time scores = ParProx.cross_validate(U, X, X_unpen, y, group_to_variables, lambdas, 5; T =Float64, criteria=auc)

20	(-0.6931299669675716, Inf, 5.0)
40	(-0.6931127550247755, 1.0165857380108762e-5, 5.0)
  0.777548 seconds (1.81 M allocations: 106.839 MiB)
20	(-0.6930909832098205, Inf, 35.0)
40	(-0.6930692158627235, 1.2856737865812443e-5, 35.0)
  0.033310 seconds (2.56 k allocations: 14.532 MiB)
20	(-0.6930194596702821, Inf, 231.0)
40	(-0.6929697871677261, 2.9340454231704357e-5, 231.0)
  0.036784 seconds (2.44 k allocations: 14.522 MiB)
20	(-0.6928195220455069, Inf, 1272.0)
40	(-0.6926704275530877, 8.808241107792448e-5, 1272.0)
  0.036732 seconds (2.44 k allocations: 14.522 MiB)
20	(-0.6921737583844234, Inf, 6412.0)
40	(-0.6916903180042857, 0.0002857735691884926, 6366.0)
  0.039209 seconds (2.56 k allocations: 14.533 MiB)
20	(-0.6904786105516274, Inf, 14137.0)
40	(-0.6893248921581476, 0.000682946423648488, 14063.0)
60	(-0.6882256525617884, 0.0006511212495149772, 13960.0)
80	(-0.6871776821871006, 0.0006211381206330579, 13885.0)
100	(-0.6861777618018515, 0.0005930100656650706, 13770.0)
120	(-0.6852231

40	(-0.6844307802351707, 0.0009683645616559671, 15399.0)
60	(-0.6828845301419246, 0.0009188093808882105, 15192.0)
80	(-0.6814173417457926, 0.0008725902604338924, 15067.0)
100	(-0.6800241402201878, 0.0008292747063874045, 14969.0)
120	(-0.6787002374980747, 0.0007886474860372696, 14864.0)
140	(-0.677441478086922, 0.0007504043673632676, 14793.0)
160	(-0.6762437862934924, 0.0007145093113681216, 14749.0)
180	(-0.6751036999787957, 0.0006806064094486054, 14646.0)
200	(-0.6740175792526096, 0.0006488108247172616, 14622.0)
220	(-0.6729825204333719, 0.0006186907553401022, 14530.0)
240	(-0.6719946136822349, 0.0005908552234874468, 14428.0)
260	(-0.6710512484487431, 0.0005645339928189204, 14307.0)
280	(-0.6701494157020425, 0.000539971297311477, 14188.0)
300	(-0.6692870275562096, 0.0005166206479752917, 14068.0)
320	(-0.6684611420487239, 0.0004949983470826485, 13939.0)
  0.261328 seconds (19.87 k allocations: 116.209 MiB, 2.27% gc time)
20	(-0.6669427502533459, Inf, 16722.0)
40	(-0.665505513385753, 0.0

120	(-0.6805520760160908, 0.0007993808592679364, 14864.0)
140	(-0.6792750927625997, 0.000760437202334933, 14797.0)
160	(-0.6780605418323331, 0.0007237825453784619, 14739.0)
180	(-0.6769045690801553, 0.0006893491576637061, 14636.0)
200	(-0.6758032537618935, 0.0006571865257985842, 14553.0)
220	(-0.6747535051427538, 0.0006268078352523151, 14440.0)
240	(-0.6737520841274502, 0.0005983090475586132, 14334.0)
260	(-0.6727961208180352, 0.0005714762830437254, 14238.0)
280	(-0.6718830491383881, 0.0005461337024247022, 14160.0)
300	(-0.6710106252292528, 0.0005220935737710479, 14089.0)
320	(-0.6701763786795051, 0.0004994960774186689, 14023.0)
  0.315533 seconds (19.87 k allocations: 116.210 MiB)
20	(-0.6686234551984976, Inf, 16884.0)
40	(-0.6671546652723159, 0.0008810159949627698, 16804.0)
60	(-0.6657641899411175, 0.0008347371972545259, 16688.0)
80	(-0.664446613244097, 0.0007916004553924713, 16627.0)
100	(-0.6631969745011138, 0.0007513474123280345, 16516.0)
120	(-0.6620108606273551, 0.00071366192716

100	(-0.6770190750608887, 0.0007308088498017223, 16313.0)
120	(-0.6758563160100691, 0.0006938297989578613, 16230.0)
140	(-0.6747521843720313, 0.0006592806078063454, 16165.0)
160	(-0.6737028370640707, 0.0006269615398401872, 16057.0)
180	(-0.6727046439488229, 0.0005967539570472954, 15987.0)
200	(-0.671754256896371, 0.0005684968640165311, 15862.0)
220	(-0.6708489736780814, 0.0005418103207118838, 15830.0)
240	(-0.6699859847698667, 0.0005167641621457103, 15767.0)
260	(-0.6691625925518726, 0.0004932965917569906, 15668.0)
  0.207013 seconds (16.09 k allocations: 94.416 MiB)
20	(-0.667833518449433, Inf, 17825.0)
40	(-0.666575907003639, 0.0007546079602548839, 17755.0)
60	(-0.6653846856441733, 0.0007152830032209244, 17711.0)
80	(-0.6642552170497593, 0.000678663093762869, 17642.0)
100	(-0.6631832016960988, 0.000644556385951527, 17603.0)
120	(-0.6621648016072995, 0.0006126950154488242, 17539.0)
140	(-0.6611963935557498, 0.0005829581952540165, 17448.0)
160	(-0.6602745744370726, 0.000555220885069398

140	(-0.657971996417883, 0.0005843357899288271, 17401.0)
160	(-0.657047208005836, 0.0005580941855964608, 17327.0)
180	(-0.6561636308818575, 0.0005335083487542204, 17280.0)
200	(-0.6553187845328041, 0.0005103828682109716, 17203.0)
220	(-0.6545100302588949, 0.0004888179939185372, 17119.0)
  0.169252 seconds (13.53 k allocations: 79.882 MiB)
20	(-0.6533415630937076, Inf, 18759.0)
40	(-0.6522298944416304, 0.0006728292810928247, 18674.0)
60	(-0.6511709930943246, 0.0006413032640074398, 18635.0)
80	(-0.6501612566062854, 0.0006119017059676748, 18587.0)
100	(-0.6491973719439718, 0.0005844568265213197, 18556.0)
120	(-0.6482763058611476, 0.0005588056320101916, 18547.0)
140	(-0.6473952524900547, 0.0005348160192650659, 18492.0)
160	(-0.6465515604495002, 0.0005123994054119581, 18441.0)
180	(-0.6457428964251424, 0.0004913671668365886, 18389.0)
  0.141764 seconds (11.09 k allocations: 65.360 MiB, 4.21% gc time)
20	(-0.6446783255038053, Inf, 19504.0)
40	(-0.6436620706076615, 0.0006182870033425826, 1947

31×5 Array{Float64,2}:
 0.515088  0.722807  0.579659  0.523509  0.576491
 0.73193   0.719298  0.707681  0.674386  0.792281
 0.807018  0.728421  0.711238  0.729123  0.813333
 0.821053  0.738947  0.716927  0.764211  0.814035
 0.847018  0.738947  0.731863  0.797193  0.823158
 0.857544  0.738246  0.737553  0.812632  0.825965
 0.859649  0.740351  0.741821  0.815439  0.825965
 0.860351  0.743158  0.739687  0.81614   0.82386
 0.860351  0.740351  0.743243  0.81614   0.820351
 0.863158  0.740351  0.745377  0.818246  0.818246
 0.863158  0.741754  0.746088  0.820351  0.818947
 0.864561  0.742456  0.747511  0.821053  0.819649
 0.864561  0.741754  0.751067  0.820351  0.818947
 ⋮                                       
 0.866667  0.745965  0.756757  0.816842  0.815439
 0.865965  0.745263  0.756046  0.817544  0.81614
 0.865965  0.745263  0.756046  0.817544  0.815439
 0.866667  0.745263  0.756046  0.818246  0.815439
 0.867368  0.745965  0.756757  0.817544  0.81614
 0.868772  0.746667  0.757468  0.81614

We select the lambda with the highest mean AUC.

In [45]:
lambda_idx = argmax(mean(scores; dims=2)[:])
lambda = lambdas[lambda_idx]

1.2589254117941663e-9

Now fitting with the selected lambda:

In [46]:
U = ParProx.LogisticUpdate(; maxiter=30000, step=20, tol=1e-5, verbose=true)
V = ParProx.LogisticVariables{Float64}(X, X_unpen, y, lambda, group_to_variables)
@time ParProx.fit!(U, V)

20	(-0.6882550177051432, Inf, 21956.0)
40	(-0.6837195236951092, 0.0026937348805460793, 21956.0)
60	(-0.679511785655015, 0.0025053340357794455, 21965.0)
80	(-0.6756051154510824, 0.0023314981363499548, 21965.0)
100	(-0.6719749218217413, 0.002171201004250507, 21965.0)
120	(-0.6685985792479504, 0.0020234600555111815, 21965.0)
140	(-0.6654552969825821, 0.0018873411199106962, 21965.0)
160	(-0.6625259907792307, 0.0017619611480350075, 21965.0)
180	(-0.6597931556525485, 0.0016464913819986165, 21965.0)
200	(-0.657240747045071, 0.0015401555941878507, 21965.0)
220	(-0.6548540665157275, 0.0014422302108901696, 21965.0)
240	(-0.6526196543991636, 0.0013520425650367207, 21965.0)
260	(-0.6505251896861824, 0.0012689686446890043, 21965.0)
280	(-0.6485593943226994, 0.001192432235230872, 21965.0)
300	(-0.6467119433282783, 0.0011219029544943168, 21965.0)
320	(-0.6449733910173964, 0.0010568878015751134, 21965.0)
340	(-0.6433350875183512, 0.0009969381847248187, 21965.0)
360	(-0.6417891146766034, 0.000941639110

2840	(-0.5846590651948506, 0.0001719641902114774, 21965.0)
2860	(-0.5843874368253931, 0.00017144062313555384, 21965.0)
2880	(-0.5841166778718435, 0.00017092109270214965, 21965.0)
2900	(-0.5838467878147979, 0.00017040161909722803, 21965.0)
2920	(-0.5835777545108944, 0.00016988954482161282, 21965.0)
2940	(-0.5833095708293468, 0.00016938170935656532, 21965.0)
2960	(-0.5830422308916688, 0.0001688773252292393, 21965.0)
2980	(-0.5827757299526832, 0.000168375679473921, 21965.0)
3000	(-0.5825100597336647, 0.00016787900802557221, 21965.0)
3020	(-0.5822452141572677, 0.00016738592351376157, 21965.0)
3040	(-0.5819811872417167, 0.00016689636873078097, 21965.0)
3060	(-0.5817179733777046, 0.00016641011131079473, 21965.0)
3080	(-0.581455567140997, 0.00016592703719268997, 21965.0)
3100	(-0.5811939632913136, 0.0001654470329110641, 21965.0)
3120	(-0.5809331564401402, 0.00016497019504646679, 21965.0)
3140	(-0.5806731396585937, 0.00016449750111062445, 21965.0)
3160	(-0.5804139074755897, 0.00016402803200967

5660	(-0.5528689878064046, 0.0001222574927134961, 21965.0)
5680	(-0.5526795328312396, 0.00012201807981555583, 21965.0)
5700	(-0.5524904709506936, 0.00012177973654823152, 21965.0)
5720	(-0.5523018003777425, 0.00012154245579385474, 21965.0)
5740	(-0.5521135193374072, 0.00012130623049767626, 21965.0)
5760	(-0.5519256264921836, 0.00012107077943436935, 21965.0)
5780	(-0.5517381197109461, 0.0001208366146681992, 21965.0)
5800	(-0.5515509972068453, 0.00012060351508758974, 21965.0)
5820	(-0.5513642572520161, 0.00012037144336429783, 21965.0)
5840	(-0.5511778981301064, 0.00012014039275206514, 21965.0)
5860	(-0.5509919181361739, 0.00011991035656461847, 21965.0)
5880	(-0.5508063156955564, 0.00011968125144902367, 21965.0)
5900	(-0.5506210891464346, 0.00011945313424296804, 21965.0)
5920	(-0.5504362370964513, 0.00011922583177589194, 21965.0)
5940	(-0.5502517574686864, 0.00011899978624513, 21965.0)
5960	(-0.5500676486137247, 0.00011877472259118988, 21965.0)
5980	(-0.54988390889297, 0.000118550634470380

8500	(-0.5292245876162524, 9.642104924190586e-5, 21965.0)
8520	(-0.5290773626921534, 9.628350251664173e-5, 21965.0)
8540	(-0.5289303615040716, 9.614642483602213e-5, 21965.0)
8560	(-0.5287835832768776, 9.600981381510357e-5, 21965.0)
8580	(-0.5286370272394355, 9.58736670841572e-5, 21965.0)
8600	(-0.5284906926245738, 9.573798229050116e-5, 21965.0)
8620	(-0.5283445788831749, 9.560261698688192e-5, 21965.0)
8640	(-0.5281986851529057, 9.54677763347172e-5, 21965.0)
8660	(-0.5280530119548245, 9.533255518070836e-5, 21965.0)
8680	(-0.5279075577377208, 9.519830984999044e-5, 21965.0)
8700	(-0.5277623211562253, 9.506490602906052e-5, 21965.0)
8720	(-0.5276173014663006, 9.493195042079824e-5, 21965.0)
8740	(-0.5274724980366184, 9.479936946051546e-5, 21965.0)
8760	(-0.5273279101029963, 9.466725034338108e-5, 21965.0)
8780	(-0.5271835368523135, 9.453562535149331e-5, 21965.0)
8800	(-0.5270393775555098, 9.44044396775416e-5, 21965.0)
8820	(-0.5268954314872091, 9.42736911331902e-5, 21965.0)
8840	(-0.526751697

11420	(-0.5097653820777172, 8.032488653503002e-5, 21958.0)
11440	(-0.5096442531381797, 8.023674404457931e-5, 21958.0)
11460	(-0.509523266482155, 8.014891768228993e-5, 21958.0)
11480	(-0.5094024217157916, 8.006133064630068e-5, 21958.0)
11500	(-0.5092817184468846, 7.997398194898721e-5, 21958.0)
11520	(-0.5091611562848667, 7.988687060746762e-5, 21958.0)
11540	(-0.5090407348407986, 7.979999564478394e-5, 21958.0)
11560	(-0.5089204537273603, 7.971335608924496e-5, 21958.0)
11580	(-0.5088003125588417, 7.962695097457776e-5, 21958.0)
11600	(-0.5086803109511341, 7.95407793397852e-5, 21958.0)
11620	(-0.508560448521721, 7.945484022900322e-5, 21958.0)
11640	(-0.5084407248896692, 7.936913269202053e-5, 21958.0)
11660	(-0.5083211397180086, 7.928362767819428e-5, 21958.0)
11680	(-0.5082016926006145, 7.919837113305765e-5, 21958.0)
11700	(-0.5080823831471089, 7.911335271798016e-5, 21958.0)
11720	(-0.5079632109827995, 7.902856213028925e-5, 21958.0)
11740	(-0.5078441759940432, 7.894382632598181e-5, 21958.0)


...

17220	(-0.4793799443814039, 6.197551296489763e-5, 21958.0)
17240	(-0.47928833200903315, 6.193003107537835e-5, 21958.0)
17260	(-0.4791967924544267, 6.188463568431242e-5, 21958.0)
17280	(-0.47910532564468145, 6.183928092165356e-5, 21958.0)
17300	(-0.47901393137612863, 6.179405522419722e-5, 21958.0)
17320	(-0.47892260950575427, 6.174891761569619e-5, 21958.0)
17340	(-0.47883135989447173, 6.170386546918478e-5, 21958.0)
17360	(-0.4787401824036158, 6.165889852787053e-5, 21958.0)
17380	(-0.47864907689494, 6.161401653671547e-5, 21958.0)
17400	(-0.47855804323061735, 6.156921923995894e-5, 21958.0)
17420	(-0.47846708127323545, 6.15245063850602e-5, 21958.0)
17440	(-0.47837619088579775, 6.147987771856933e-5, 21958.0)
17460	(-0.4782853719317203, 6.143533298901876e-5, 21958.0)
17480	(-0.4781946242748304, 6.139087194587268e-5, 21958.0)
17500	(-0.47810394777936494, 6.13464943393774e-5, 21958.0)
17520	(-0.47801334230996945, 6.130219992052446e-5, 21958.0)
17540	(-0.4779228077316957, 6.125798844168961e-5, 

20000	(-0.4672887278488509, 5.6380195864693726e-5, 21958.0)
20020	(-0.4672060586765851, 5.6344623017955444e-5, 21958.0)
20040	(-0.46712344626261704, 5.630910894274961e-5, 21950.0)
20060	(-0.4670408905085784, 5.6273655746588934e-5, 21950.0)
20080	(-0.4669583913256358, 5.6238256947432704e-5, 21950.0)
20100	(-0.4668759486198975, 5.6202916010643444e-5, 21950.0)
20120	(-0.46679356229772073, 5.616763278379649e-5, 21950.0)
20140	(-0.46671123226571404, 5.613240711296281e-5, 21950.0)
20160	(-0.4666289584307336, 5.609723884660728e-5, 21950.0)
20180	(-0.4665467406998836, 5.6062127832864173e-5, 21950.0)
20200	(-0.46646457898051585, 5.602707392010504e-5, 21950.0)
20220	(-0.46638247318022746, 5.599207695815032e-5, 21950.0)
20240	(-0.4663004232068612, 5.5957136796566316e-5, 21950.0)
20260	(-0.46621842896850424, 5.592225328572531e-5, 21950.0)
20280	(-0.4661364903734878, 5.5887426276086595e-5, 21950.0)
20300	(-0.46605460733038445, 5.585265562001415e-5, 21950.0)
20320	(-0.46597277974800966, 5.5817941168

22800	(-0.456228722521877, 5.190463027802319e-5, 21955.0)
22820	(-0.45615318322330434, 5.187592860626657e-5, 21955.0)
22840	(-0.45607768957304956, 5.184726803754044e-5, 21955.0)
22860	(-0.45600224158379676, 5.181859416007968e-5, 21955.0)
22880	(-0.4559268392292057, 5.178993377922774e-5, 21955.0)
22900	(-0.4558514823247327, 5.17613955735622e-5, 21955.0)
22920	(-0.45577617081840727, 5.1732888499685906e-5, 21955.0)
22940	(-0.45570090467253505, 5.170440275926932e-5, 21955.0)
22960	(-0.45562568377973994, 5.167598623279358e-5, 21955.0)
22980	(-0.45555050807456354, 5.1647610137448515e-5, 21955.0)
23000	(-0.4554753774916999, 5.161927437968871e-5, 21958.0)
23020	(-0.45540029196595055, 5.1590978896903965e-5, 21958.0)
23040	(-0.4553252514322925, 5.156272357961217e-5, 21958.0)
23060	(-0.4552502558258802, 5.1534508317107324e-5, 21958.0)
23080	(-0.4551753050820211, 5.150633301523688e-5, 21958.0)
23100	(-0.4551003991361742, 5.147819758095557e-5, 21958.0)
23120	(-0.45502553792395006, 5.145010192121939

25680	(-0.4457914290233174, 4.8150072157378344e-5, 21957.0)
25700	(-0.445721851605219, 4.8126420736588194e-5, 21957.0)
25720	(-0.4456523116823322, 4.810279921722634e-5, 21957.0)
25740	(-0.4455828092064586, 4.807920752168476e-5, 21957.0)
25760	(-0.44551334477781945, 4.80551970620653e-5, 21957.0)
25780	(-0.445443917816592, 4.803158418788229e-5, 21957.0)
25800	(-0.4453745281606782, 4.800807995565055e-5, 21957.0)
25820	(-0.4453051757623325, 4.798460526451242e-5, 21957.0)
25840	(-0.44523586057391834, 4.796116004664185e-5, 21957.0)
25860	(-0.4451665825524613, 4.793774108359541e-5, 21957.0)
25880	(-0.4450973417034248, 4.7914314861903886e-5, 21957.0)
25900	(-0.4450281379222899, 4.789095749678632e-5, 21957.0)
25920	(-0.4449589711618404, 4.7867629344434606e-5, 21957.0)
25940	(-0.44488984137496274, 4.784433034139895e-5, 21957.0)
25960	(-0.44482074851464576, 4.782106042428333e-5, 21957.0)
25980	(-0.4447516925339788, 4.7797819531090454e-5, 21957.0)
26000	(-0.44468267338615197, 4.777460759949295e-5,

28500	(-0.43633100233991634, 4.508049730414324e-5, 21963.0)
28520	(-0.4362662834590217, 4.506050280508494e-5, 21963.0)
28540	(-0.4362015961767984, 4.504053079700442e-5, 21963.0)
28560	(-0.43613694045700335, 4.5020581236808364e-5, 21963.0)
28580	(-0.4360723162634623, 4.500065408208432e-5, 21963.0)
28600	(-0.43600772356007045, 4.49807492899412e-5, 21963.0)
28620	(-0.43594316231079117, 4.496086681828495e-5, 21963.0)
28640	(-0.43587863247965647, 4.494100662481355e-5, 21963.0)
28660	(-0.4358141340307669, 4.4921168667210304e-5, 21963.0)
28680	(-0.4357496669282911, 4.4901352903491866e-5, 21963.0)
28700	(-0.4356852311364663, 4.488155929123501e-5, 21963.0)
28720	(-0.43562082661959683, 4.486178778912323e-5, 21963.0)
28740	(-0.4355564533420545, 4.4842038355554994e-5, 21963.0)
28760	(-0.43549211126827975, 4.482231094806356e-5, 21963.0)
28780	(-0.43542780036277917, 4.48026055259079e-5, 21963.0)
28800	(-0.4353635205901275, 4.4782922046979186e-5, 21963.0)
28820	(-0.4352992719149654, 4.476326047066253

In order to obtain the results in the original dimension, we need to obtain the "group matrix". Multiplying the group matrix by the _penalized_ part of the coefficients gives the aggregate coefficient for each penalized variable. Non-penalized variables (in this case, the last two variables) should be treated separately. 

In [47]:
_, grpmat, _ = ParProx.mapper_mat_idx(group_to_variables, length(gene_names));

In [48]:
size(grpmat)

(1043, 21963)

In [49]:
β_orig = vcat(grpmat * collect(V.β[1:end-2]), collect(V.β)[end-1:end]);

The following prints the coefficients in the replicated dimensions:

In [50]:
for (v, β) in zip(variable_names_replicated[V.β .!= 0], V.β[V.β .!= 0])
    println("$v\t$β")
end

KEGG:00280: Valine, leucine and isoleucine degradation	ABAT	4-aminobutyrate aminotransferase	0.0008323544134017853
KEGG:00280: Valine, leucine and isoleucine degradation	AUH	AU RNA binding protein/enoyl-CoA hydratase	-0.001459342688879147
KEGG:00280: Valine, leucine and isoleucine degradation	HMGCL	3-hydroxymethyl-3-methylglutaryl-CoA lyase	-0.0010615725170478396
KEGG:00280: Valine, leucine and isoleucine degradation	IVD	isovaleryl-CoA dehydrogenase	-0.0009348559171503025
KEGG:00280: Valine, leucine and isoleucine degradation	MCCC1	methylcrotonoyl-CoA carboxylase 1 (alpha)	8.306605556755142e-5
KEGG:00280: Valine, leucine and isoleucine degradation	MCCC2	methylcrotonoyl-CoA carboxylase 2 (beta)	-0.0007834944198441149
KEGG:00280: Valine, leucine and isoleucine degradation	MUT	methylmalonyl CoA mutase	0.002754888016414541
KEGG:04727: GABAergic synapse	ABAT	4-aminobutyrate aminotransferase	0.0008232735329612728
KEGG:04727: GABAergic synapse	GABRA4	gamma-aminobutyric acid (GABA) A receptor,

KEGG:00650: Butanoate metabolism	ABAT	4-aminobutyrate aminotransferase	0.0008342785686320111
KEGG:00650: Butanoate metabolism	ALDH5A1	aldehyde dehydrogenase 5 family, member A1	0.002331490731821373
KEGG:00650: Butanoate metabolism	BDH1	3-hydroxybutyrate dehydrogenase, type 1	-0.0002073436385173155
KEGG:00650: Butanoate metabolism	HMGCL	3-hydroxymethyl-3-methylglutaryl-CoA lyase	-0.0010643922027909467
GO:0001666: response to hypoxia	ABAT	4-aminobutyrate aminotransferase	0.0008334434534379154
GO:0001666: response to hypoxia	ADM	adrenomedullin	0.002238812559897346
GO:0001666: response to hypoxia	AGER	advanced glycosylation end product-specific receptor	-0.00017426454237396907
GO:0001666: response to hypoxia	BNIP3	BCL2/adenovirus E1B 19kDa interacting protein 3	0.002914150843654699
GO:0001666: response to hypoxia	CAV1	caveolin 1, caveolae protein, 22kDa	-0.0005354719528487098
GO:0001666: response to hypoxia	CCL2	chemokine (C-C motif) ligand 2	0.0008172213365582931
GO:0001666: response to h

GO:0035094: response to nicotine	BCL2	B-cell CLL/lymphoma 2	-5.08663682741707e-6
GO:0035094: response to nicotine	CHRNB2	cholinergic receptor, nicotinic, beta 2 (neuronal)	-0.000758327813289944
GO:0035094: response to nicotine	DRD2	dopamine receptor D2	-0.00023001114993561648
GO:0035094: response to nicotine	GPX1	glutathione peroxidase 1	0.0003867305456564964
GO:0035094: response to nicotine	HDAC2	histone deacetylase 2	-0.00118854149922459
GO:0035094: response to nicotine	HMOX1	heme oxygenase (decycling) 1	-0.0002107114039207381
GO:0035094: response to nicotine	PDX1	pancreatic and duodenal homeobox 1	0.0002469304872419897
GO:0035094: response to nicotine	STAR	steroidogenic acute regulatory protein	0.0008692233019067945
GO:0042135: neurotransmitter catabolic process	ABAT	4-aminobutyrate aminotransferase	0.0008384285830425878
GO:0042135: neurotransmitter catabolic process	ALDH5A1	aldehyde dehydrogenase 5 family, member A1	0.0023452776173856025
GO:0042493: response to drug	ABAT	4-aminobut

GO:0006355: regulation of transcription, DNA-templated	CEBPB	CCAAT/enhancer binding protein (C/EBP), beta	-0.0002745449759790138
GO:0006355: regulation of transcription, DNA-templated	CELSR2	cadherin, EGF LAG seven-pass G-type receptor 2	-0.0011360981432418863
GO:0006355: regulation of transcription, DNA-templated	CHAF1B	chromatin assembly factor 1, subunit B (p60)	0.006292511066352083
GO:0006355: regulation of transcription, DNA-templated	CHD3	chromodomain helicase DNA binding protein 3	0.00011917125375918101
GO:0006355: regulation of transcription, DNA-templated	CLOCK	clock circadian regulator	0.00020035738705752516
GO:0006355: regulation of transcription, DNA-templated	DAXX	death-domain associated protein	0.0008926960114535764
GO:0006355: regulation of transcription, DNA-templated	DLX6	distal-less homeobox 6	-0.00018007477071055246
GO:0006355: regulation of transcription, DNA-templated	E2F1	E2F transcription factor 1	0.0021001730920734304
GO:0006355: regulation of transcription, DNA

GO:0032367: intracellular cholesterol transport	ABCG1	ATP-binding cassette, sub-family G (WHITE), member 1	-0.0023414183670599823
GO:0032367: intracellular cholesterol transport	STAR	steroidogenic acute regulatory protein	0.0008955133587820492
GO:0032367: intracellular cholesterol transport	VPS4A	vacuolar protein sorting 4 homolog A (S. cerevisiae)	-0.0006857834708294202
GO:0033344: cholesterol efflux	ABCG1	ATP-binding cassette, sub-family G (WHITE), member 1	-0.0023351541238514494
GO:0033344: cholesterol efflux	CAV1	caveolin 1, caveolae protein, 22kDa	-0.0005351373609785363
GO:0033344: cholesterol efflux	SOAT1	sterol O-acyltransferase 1	-0.00040815997631377267
GO:0033700: phospholipid efflux	ABCG1	ATP-binding cassette, sub-family G (WHITE), member 1	-0.0023606226924747847
GO:0033993: response to lipid	ABCG1	ATP-binding cassette, sub-family G (WHITE), member 1	-0.002348053988443846
GO:0033993: response to lipid	GATA2	GATA binding protein 2	-0.00045468612400768345
GO:0033993: response t

...

KEGG:04022: cGMP-PKG signaling pathway	CREB1	cAMP responsive element binding protein 1	0.0006489619781641577
KEGG:04022: cGMP-PKG signaling pathway	GATA4	GATA binding protein 4	-0.0010834492017024245
KEGG:04022: cGMP-PKG signaling pathway	INSR	insulin receptor	-0.001348771581336535
KEGG:04022: cGMP-PKG signaling pathway	ITPR1	inositol 1,4,5-trisphosphate receptor, type 1	-0.0007704561243640072
KEGG:04022: cGMP-PKG signaling pathway	KCNMA1	potassium large conductance calcium-activated channel, subfamily M, alpha member 1	-0.0006479979619600895
KEGG:04022: cGMP-PKG signaling pathway	MEF2C	myocyte enhancer factor 2C	0.0009623584081566341
KEGG:04022: cGMP-PKG signaling pathway	PIK3CD	phosphatidylinositol-4,5-bisphosphate 3-kinase, catalytic subunit delta	-0.0007731994922275429
KEGG:04022: cGMP-PKG signaling pathway	PLCB1	phospholipase C, beta 1 (phosphoinositide-specific)	0.00015005615254737393
KEGG:04022: cGMP-PKG signaling pathway	RAF1	Raf-1 proto-oncogene, serine/threonine kinase	0.0014

GO:0006915: apoptotic process	DAXX	death-domain associated protein	0.000891037885543598
GO:0006915: apoptotic process	DHCR24	24-dehydrocholesterol reductase	-0.0017067530777823046
GO:0006915: apoptotic process	DIABLO	diablo, IAP-binding mitochondrial protein	-0.0005955950874973306
GO:0006915: apoptotic process	DOCK1	dedicator of cytokinesis 1	-0.0006838636074784749
GO:0006915: apoptotic process	DSG2	desmoglein 2	1.6684566336291924e-5
GO:0006915: apoptotic process	DSP	desmoplakin	-5.6663577730001117e-5
GO:0006915: apoptotic process	E2F1	E2F transcription factor 1	0.0020967558273057426
GO:0006915: apoptotic process	EDAR	ectodysplasin A receptor	0.0005937664990297259
GO:0006915: apoptotic process	ESPL1	extra spindle pole bodies homolog 1 (S. cerevisiae)	0.00019668379559849639
GO:0006915: apoptotic process	FAS	Fas cell surface death receptor	-3.3268354249467116e-5
GO:0006915: apoptotic process	FEM1B	fem-1 homolog b (C. elegans)	-0.0041294375883662495
GO:0006915: apoptotic process	FGF8	fibr

GO:0006805: xenobiotic metabolic process	UGDH	UDP-glucose 6-dehydrogenase	0.0005021831791150158
GO:0006805: xenobiotic metabolic process	UGP2	UDP-glucose pyrophosphorylase 2	0.0007852680716861067
GO:0007586: digestion	AKR1C1	aldo-keto reductase family 1, member C1	-0.0014971109160566132
GO:0007586: digestion	NMU	neuromedin U	0.0018630596659245915
GO:0007586: digestion	PGC	progastricsin (pepsinogen C)	0.0013304868788324748
GO:0007586: digestion	SLC15A1	solute carrier family 15 (oligopeptide transporter), member 1	-0.002777325068263748
GO:0007603: phototransduction, visible light	AKR1C1	aldo-keto reductase family 1, member C1	-0.0014807883341618933
GO:0007603: phototransduction, visible light	LRP8	low density lipoprotein receptor-related protein 8, apolipoprotein e receptor	-0.0009278360939371801
GO:0007603: phototransduction, visible light	METAP1	methionyl aminopeptidase 1	-0.0010119364799306915
GO:0007603: phototransduction, visible light	PRKCA	protein kinase C, alpha	-6.42321874417016

Excessive output truncated after 524293 bytes.

KEGG:05166: HTLV-I infection	E2F1	E2F transcription factor 1	0.002096507746263104
KEGG:05166: HTLV-I infection	E2F3	E2F transcription factor 3	0.0021608927453620713
KEGG:05166: HTLV-I infection	EGR1	early growth response 1	0.001092420152729813
KEGG:05166: HTLV-I infection	EGR2	early growth response 2	0.001437952547003632
KEGG:05166: HTLV-I infection	ETS2	v-ets avian erythroblastosis virus E26 oncogene homolog 2	-0.0010148671813836451
KEGG:05166: HTLV-I infection	FZD3	frizzled class receptor 3	0.0003783187873494029
KEGG:05166: HTLV-I infection	FZD7	frizzled class receptor 7	0.0010700104194553235
KEGG:05166: HTLV-I infection	GSK3B	glycogen synthase kinase 3 beta	0.0015481661412175235
KEGG:05166: HTLV-I infection	ICAM1	intercellular adhesion molecule 1	0.0007288551674977712
KEGG:05166: HTLV-I infection	MAD2L1	MAD2 mitotic arrest deficient-like 1 (yeast)	0.00031861014213334544
KEGG:05166: HTLV-I infection	MSX1	msh homeobox 1	-0.002767385544638793
KEGG:05166: HTLV-I infection	MYC	v-myc avia

The following prints the coefficients in the original dimensions:

In [51]:
for (v, β) in zip(variable_names[β_orig .!= 0], β_orig[β_orig .!= 0])
    println("$v\t$β")
end

RFC2	0.017828370354349202
PAX8	0.03684757874519625
THRA	0.028067107007347436
EPHB3	0.019420774902517743
CFL1	-0.030460550648403972
YY1	0.013402782876598915
ZPR1	-0.0041704594130534
RHOA	0.043220992206706325
GUK1	0.05284161532839049
DSP	-0.0008924333259685115
RAD21	-0.0016314685844209583
SF3B2	0.004117778492081896
NDRG1	-0.006543628992920163
CD63	-0.015885903765799944
DNAJB1	-0.0020740235707915316
XBP1	0.021161233616516405
SPTBN1	0.03684346496032824
HMGB1	-0.030756596050312508
GPX1	0.02123400737801506
NUMA1	0.021341452238237465
ARL6IP5	0.0010924161799196028
STMN1	0.007247922509085323
ODC1	0.00984689697218672
DAZAP2	-0.0007855012627521636
TMBIM6	-0.0022818769304895872
GSTP1	0.00815519724724144
ZNF207	-0.0027205566179267678
DHCR24	-0.03582951324451187
DEK	0.01752513149401217
HIF1A	-0.04980124402515669
SNX17	0.011478189129344094
CD9	-0.03889280343675502
JUP	-0.038198768007394045
TGM2	-0.0017271186611303288
MMP2	-0.02598193263059217
THBS1	-0.08446415723750327
TUFM	-0.006526015058534709
POLD

RAD51	-0.05959231669664289
CCNE2	0.018853635423271606
ORM1	-0.0027846959939131543
CENPE	0.03231574820149772
PSPH	-0.021255067676917736
KIT	0.02330305444052017
AUH	-0.008742765931883348
PRIM1	0.007640062297202148
SLC22A5	-0.02409020140972212
SERPINF2	0.014325953337234937
CRIP1	0.01716096384814977
ORC1	-0.005887435280780701
MOK	0.005371912559628551
PEX11A	0.006381301901495135
GLI3	-0.14256324992470182
KAL1	-0.0011795132247510995
GALK2	0.00012695444364892882
ESR1	-0.04057493084726799
AREG	0.05169677369466025
GPSM2	0.0071155939118706865
CXCL13	0.0783715582690652
EGR2	0.03901184423000848
PER2	0.03325074136646314
PGC	0.005304266152110364
LRP8	-0.020463024120600196
TFAP2C	0.02018951968784407
CDC14A	0.002323084411661582
IGFBP1	0.003091742143176985
SLC15A2	-0.0010016510545315745
S100A1	0.007943911280445212
DCT	-0.030649335406504684
BARD1	0.020946437515230742
AGTR1	0.012107886892131901
INPP4B	0.0043235597350753455
PDZK1	-0.0015817353024116022
CHEK1	0.01974575498226775
MLLT10	-0.00032716360192899

METAP1	-0.008021374838937058
SLC25A44	0.00034531985411083167
JMJD6	0.07225247109460833
C16orf45	0.0015129853607803163
RCHY1	-0.012047180085251727
BBS4	-0.024491286185357838
TCF7L2	0.022737387684181265
FAM171A1	-0.0008474461993496482
OBSL1	0.0004996753392981064
NCAPD3	-0.006291945324096308
CBS	0.027408147568601843
CKAP5	-0.00784421841174049
POLD3	0.04009422205891675
AXIN1	-0.040087790721424446
DNAJC16	-0.00010774179655897064
SMC5	-0.00840435524672013
NCAPH	0.0005077986713116518
KANK1	0.035704184864029716
NEDD4	0.08611271769809645
LOC100272216	-0.0004901303998456607
DNAJC2	-0.00785480274112632
HOXA10	-0.03585348281798796
ROBO1	-0.03381507510547673
TNNT1	0.002361670276473978
FOXC1	-0.0010032433396977912
YAP1	-0.005210933274387347
ATAD2B	-0.001240321054198613
SPDEF	0.031499791704416746
SOSTDC1	-0.012182241980776623
CLCN2	0.008450306755612699
CCNE1	0.013357563921641039
DNA2	-0.0005924515358255864
DLX5	-0.06904625505431022
ELOVL2	0.006080546124612895
HIPK2	0.028394484848613795
HOXA5	0.026519

### (optional) analysis using GPU

Once again, the analysis can be carried out using a GPU. At this time, data should be converted to `CuArray`s beforehand for overlapping groups. 

In [52]:
Random.seed!(222)
T = Float64
A = CuArray
U = ParProx.LogisticUpdate(; maxiter=30000, step=20, tol=5e-4, verbose=true)
lambdas = 10 .^ (range(-6, stop=-9, length=31))

scores = ParProx.cross_validate(U, 
    adapt(A{T}, X), adapt(A{T}, X_unpen), adapt(A{Int32}, y), group_to_variables, lambdas, 5; 
    T=Float64, criteria=auc)

20	(-0.6931299669675715, Inf, 5.0)
40	(-0.6931127550247755, 1.016585738004319e-5, 5.0)
  3.943202 seconds (5.90 M allocations: 345.065 MiB, 1.30% gc time)
20	(-0.6930909832098204, Inf, 35.0)
40	(-0.6930692158627236, 1.2856737865681294e-5, 35.0)
  0.019692 seconds (28.36 k allocations: 48.852 MiB)
20	(-0.693019459670282, Inf, 231.0)
40	(-0.6929697871677261, 2.934045423163878e-5, 231.0)
  0.025621 seconds (29.73 k allocations: 48.882 MiB)
20	(-0.6928195220455068, Inf, 1272.0)
40	(-0.6926704275530874, 8.808241107805568e-5, 1272.0)
  0.022867 seconds (29.74 k allocations: 48.881 MiB)
20	(-0.6921737583844234, Inf, 6412.0)
40	(-0.6916903180042857, 0.0002857735691884926, 6366.0)
  0.051537 seconds (40.33 k allocations: 49.099 MiB, 55.96% gc time)
20	(-0.6904786105516274, Inf, 14137.0)
40	(-0.6893248921581476, 0.000682946423648488, 14063.0)
60	(-0.6882256525617884, 0.0006511212495149772, 13960.0)
80	(-0.6871776821871006, 0.0006211381206330579, 13885.0)
100	(-0.6861777618018517, 0.0005930100656

140	(-0.6774414780869222, 0.0007504043673630689, 14793.0)
160	(-0.6762437862934926, 0.0007145093113681216, 14749.0)
180	(-0.6751036999787958, 0.0006806064094486717, 14646.0)
200	(-0.6740175792526096, 0.000648810824717328, 14622.0)
220	(-0.672982520433372, 0.0006186907553400357, 14530.0)
240	(-0.6719946136822349, 0.0005908552234875133, 14428.0)
260	(-0.6710512484487431, 0.0005645339928189204, 14307.0)
280	(-0.6701494157020427, 0.0005399712973113439, 14188.0)
300	(-0.6692870275562094, 0.0005166206479755577, 14068.0)
320	(-0.6684611420487239, 0.0004949983470825154, 13939.0)
  0.152588 seconds (243.23 k allocations: 391.106 MiB, 5.32% gc time)
20	(-0.6669427502533459, Inf, 16722.0)
40	(-0.6655055133857531, 0.0008629433262403741, 16610.0)
60	(-0.6641438612476948, 0.0008182298236148214, 16527.0)
80	(-0.6628526067344715, 0.0007765297465293606, 16375.0)
100	(-0.6616267403085259, 0.0007377507813325587, 16266.0)
120	(-0.660462179099486, 0.0007013476270031171, 16238.0)
140	(-0.6593549963715324, 0

140	(-0.6792750927625997, 0.000760437202334999, 14797.0)
160	(-0.678060541832333, 0.000723782545378528, 14739.0)
180	(-0.6769045690801553, 0.0006893491576636399, 14636.0)
200	(-0.6758032537618935, 0.0006571865257985842, 14553.0)
220	(-0.6747535051427538, 0.0006268078352523151, 14440.0)
240	(-0.6737520841274502, 0.0005983090475586132, 14334.0)
260	(-0.6727961208180353, 0.000571476283043659, 14238.0)
280	(-0.6718830491383884, 0.0005461337024246359, 14160.0)
300	(-0.6710106252292527, 0.0005220935737712472, 14089.0)
320	(-0.6701763786795051, 0.0004994960774186024, 14023.0)
  0.151872 seconds (243.22 k allocations: 391.124 MiB, 5.42% gc time)
20	(-0.6686234551984978, Inf, 16884.0)
40	(-0.6671546652723159, 0.0008810159949628364, 16804.0)
60	(-0.6657641899411175, 0.0008347371972545259, 16688.0)
80	(-0.6644466132440972, 0.0007916004553924046, 16627.0)
100	(-0.6631969745011139, 0.0007513474123280344, 16516.0)
120	(-0.662010860627355, 0.0007136619271616359, 16392.0)
140	(-0.6608838013001387, 0.0

20	(-0.6823440080098861, Inf, 16683.0)
40	(-0.6809019339989018, 0.0008579168015790608, 16623.0)
60	(-0.6795373182116263, 0.0008124950678252686, 16519.0)
80	(-0.6782446554422296, 0.0007702469155524541, 16365.0)
100	(-0.6770190750608887, 0.0007308088498017885, 16313.0)
120	(-0.675856316010069, 0.0006938297989579277, 16230.0)
140	(-0.6747521843720312, 0.0006592806078063454, 16165.0)
160	(-0.6737028370640706, 0.0006269615398401873, 16057.0)
180	(-0.6727046439488228, 0.0005967539570472955, 15987.0)
200	(-0.671754256896371, 0.0005684968640164646, 15862.0)
220	(-0.6708489736780814, 0.0005418103207118838, 15830.0)
240	(-0.6699859847698665, 0.0005167641621458433, 15767.0)
260	(-0.6691625925518724, 0.0004932965917569906, 15668.0)
  0.134343 seconds (218.82 k allocations: 318.463 MiB, 10.85% gc time)
20	(-0.6678335184494327, Inf, 17825.0)
40	(-0.666575907003639, 0.0007546079602547506, 17755.0)
60	(-0.6653846856441733, 0.0007152830032209244, 17711.0)
80	(-0.6642552170497591, 0.0006786630937630026,

180	(-0.6561636308818576, 0.0005335083487541533, 17280.0)
200	(-0.6553187845328039, 0.000510382868211173, 17203.0)
220	(-0.654510030258895, 0.0004888179939183358, 17119.0)
  0.103970 seconds (172.71 k allocations: 269.012 MiB, 7.10% gc time)
20	(-0.6533415630937076, Inf, 18759.0)
40	(-0.6522298944416304, 0.0006728292810928247, 18674.0)
60	(-0.6511709930943247, 0.0006413032640073726, 18635.0)
80	(-0.6501612566062855, 0.0006119017059676747, 18587.0)
100	(-0.6491973719439719, 0.0005844568265213197, 18556.0)
120	(-0.6482763058611475, 0.0005588056320103262, 18547.0)
140	(-0.6473952524900548, 0.000534816019264931, 18492.0)
160	(-0.6465515604495004, 0.0005123994054119581, 18441.0)
180	(-0.6457428964251424, 0.0004913671668366561, 18389.0)
  0.077947 seconds (126.86 k allocations: 219.792 MiB)
20	(-0.6446783255038053, Inf, 19504.0)
40	(-0.6436620706076615, 0.0006182870033425826, 19475.0)
60	(-0.6426907122108307, 0.0005913215370429659, 19444.0)
80	(-0.6417611408924171, 0.0005662037523365652, 194

31×5 Array{Float64,2}:
 0.515088  0.722807  0.579659  0.523509  0.576491
 0.73193   0.719298  0.707681  0.674386  0.792281
 0.807018  0.728421  0.711238  0.729123  0.813333
 0.821053  0.738947  0.716927  0.764211  0.814035
 0.847018  0.738947  0.731863  0.797193  0.823158
 0.857544  0.738246  0.737553  0.812632  0.825965
 0.859649  0.740351  0.741821  0.815439  0.825965
 0.860351  0.743158  0.739687  0.81614   0.82386
 0.860351  0.740351  0.743243  0.81614   0.820351
 0.863158  0.740351  0.745377  0.818246  0.818246
 0.863158  0.741754  0.746088  0.820351  0.818947
 0.864561  0.742456  0.747511  0.821053  0.819649
 0.864561  0.741754  0.751067  0.820351  0.818947
 ⋮                                       
 0.866667  0.745965  0.756757  0.816842  0.815439
 0.865965  0.745263  0.756046  0.817544  0.81614
 0.865965  0.745263  0.756046  0.817544  0.815439
 0.866667  0.745263  0.756046  0.818246  0.815439
 0.867368  0.745965  0.756757  0.817544  0.81614
 0.868772  0.746667  0.757468  0.81614

In [53]:
lambda_idx = argmax(mean(scores; dims=2)[:])
lambda = lambdas[lambda_idx]

1.2589254117941663e-9

In [54]:
U = ParProx.LogisticUpdate(; maxiter=30000, step=20, tol=1e-5, verbose=true)
V = ParProx.LogisticVariables{Float64}(adapt(A{T}, X), 
    adapt(A{T}, X_unpen), adapt(A{Int}, y), lambda, group_to_variables)
@time ParProx.fit!(U, V)

20	(-0.6882550177051432, Inf, 21956.0)
40	(-0.6837195236951092, 0.0026937348805460793, 21956.0)
60	(-0.6795117856550147, 0.0025053340357796446, 21965.0)
80	(-0.6756051154510823, 0.0023314981363498225, 21965.0)
100	(-0.6719749218217413, 0.0021712010042504405, 21965.0)
120	(-0.6685985792479503, 0.002023460055511248, 21965.0)
140	(-0.6654552969825818, 0.00188734111991083, 21965.0)
160	(-0.6625259907792304, 0.0017619611480350081, 21965.0)
180	(-0.6597931556525487, 0.0016464913819982819, 21965.0)
200	(-0.6572407470450712, 0.0015401555941878505, 21965.0)
220	(-0.6548540665157275, 0.0014422302108903038, 21965.0)
240	(-0.652619654399164, 0.0013520425650364518, 21965.0)
260	(-0.6505251896861826, 0.001268968644689206, 21965.0)
280	(-0.6485593943226992, 0.0011924322352310746, 21965.0)
300	(-0.6467119433282783, 0.0011219029544941822, 21965.0)
320	(-0.6449733910173964, 0.0010568878015751134, 21965.0)
340	(-0.6433350875183513, 0.000996938184724751, 21965.0)
360	(-0.6417891146766035, 0.00094163911060

2920	(-0.5835777545108944, 0.00016988954482147258, 21965.0)
2940	(-0.5833095708293466, 0.00016938170935663544, 21965.0)
2960	(-0.5830422308916687, 0.0001688773252292393, 21965.0)
2980	(-0.5827757299526831, 0.00016837567947392102, 21965.0)
3000	(-0.5825100597336648, 0.00016787900802543192, 21965.0)
3020	(-0.5822452141572676, 0.00016738592351390192, 21965.0)
3040	(-0.5819811872417168, 0.0001668963687306406, 21965.0)
3060	(-0.5817179733777047, 0.00016641011131079473, 21965.0)
3080	(-0.5814555671409971, 0.00016592703719268994, 21965.0)
3100	(-0.5811939632913136, 0.0001654470329111343, 21965.0)
3120	(-0.5809331564401403, 0.00016497019504639656, 21965.0)
3140	(-0.5806731396585939, 0.00016449750111055417, 21965.0)
3160	(-0.5804139074755902, 0.00016402803200953563, 21965.0)
3180	(-0.5801554547311777, 0.00016356159366387482, 21965.0)
3200	(-0.5798977779690885, 0.0001630971102576303, 21965.0)
3220	(-0.5796408700486428, 0.00016263691660353528, 21965.0)
3240	(-0.5793847257453805, 0.000162179802733

5880	(-0.5508063156955564, 0.00011968125144909525, 21965.0)
5900	(-0.5506210891464345, 0.00011945313424303965, 21965.0)
5920	(-0.5504362370964512, 0.00011922583177589194, 21965.0)
5940	(-0.5502517574686863, 0.00011899978624513, 21965.0)
5960	(-0.5500676486137245, 0.00011877472259126155, 21965.0)
5980	(-0.5498839088929698, 0.00011855063447038004, 21965.0)
6000	(-0.5497005367349536, 0.00011832747919321944, 21965.0)
6020	(-0.5495175305195384, 0.00011810528878221566, 21965.0)
6040	(-0.549334888590231, 0.00011788408732832546, 21965.0)
6060	(-0.5491526093507461, 0.00011766383659347061, 21965.0)
6080	(-0.5489706912151558, 0.00011744453050141023, 21965.0)
6100	(-0.5487891326078018, 0.0001172261630273218, 21965.0)
6120	(-0.5486079327241218, 0.00011700823678546843, 21965.0)
6140	(-0.5484270895021864, 0.0001167915642663456, 21965.0)
6160	(-0.548246601143041, 0.00011657597634134199, 21965.0)
6180	(-0.548066466111183, 0.0001163613034719977, 21965.0)
6200	(-0.5478866830097613, 0.00011614745665501346

8860	(-0.5266081761528866, 9.401349675067098e-5, 21965.0)
8880	(-0.5264648654543149, 9.388404660660148e-5, 21965.0)
8900	(-0.5263217651190975, 9.375502498079494e-5, 21965.0)
8920	(-0.5261788744399157, 9.362642975526455e-5, 21965.0)
8940	(-0.5260361927129913, 9.349825882620972e-5, 21965.0)
8960	(-0.525893719238062, 9.337051010366662e-5, 21965.0)
8980	(-0.5257514533183596, 9.324318151101295e-5, 21965.0)
9000	(-0.5256093942605866, 9.311627098483632e-5, 21965.0)
9020	(-0.5254675413748927, 9.29897764760396e-5, 21965.0)
9040	(-0.5253258939748531, 9.286369594796264e-5, 21965.0)
9060	(-0.5251844513774457, 9.27380273773417e-5, 21965.0)
9080	(-0.5250432129760817, 9.261272084770507e-5, 21965.0)
9100	(-0.5249021781305354, 9.248779860698123e-5, 21965.0)
9120	(-0.5247613460580843, 9.236335431454166e-5, 21965.0)
9140	(-0.5246207160891024, 9.22393140128887e-5, 21965.0)
9160	(-0.5244802875572576, 9.211567574273528e-5, 21965.0)
9180	(-0.5243400597994896, 9.199243755783287e-5, 21965.0)
9200	(-0.524200032

11720	(-0.5079632109827995, 7.902856213021562e-5, 21958.0)
11740	(-0.5078441759940432, 7.894382632598181e-5, 21958.0)
11760	(-0.5077252777349746, 7.885936571100824e-5, 21958.0)
11780	(-0.5076065162159041, 7.877487779006966e-5, 21958.0)
11800	(-0.5074878908861946, 7.869073471614122e-5, 21958.0)
11820	(-0.5073694009989, 7.86070668649967e-5, 21958.0)
11840	(-0.5072510461883886, 7.852362140386632e-5, 21958.0)
11860	(-0.5071328260905135, 7.844039744112057e-5, 21958.0)
11880	(-0.5070147403426059, 7.835739408943606e-5, 21958.0)
11900	(-0.506896788583466, 7.82746104666835e-5, 21958.0)
11920	(-0.5067789704533558, 7.819204569512131e-5, 21958.0)
11940	(-0.5066612855939906, 7.810969890213645e-5, 21958.0)
11960	(-0.5065437336666447, 7.802755719527701e-5, 21958.0)
11980	(-0.5064263143099826, 7.794563567211721e-5, 21958.0)
12000	(-0.5063090271579861, 7.78639375333482e-5, 21958.0)
12020	(-0.5061918718581111, 7.778245392498711e-5, 21958.0)
12040	(-0.5060748480592321, 7.770118399481429e-5, 21958.0)
1206

...

17580	(-0.47774195071074366, 6.11698133177231e-5, 21958.0)
17600	(-0.47765162800018746, 6.11258491816784e-5, 21958.0)
17620	(-0.4775613756449938, 6.108196700408986e-5, 21958.0)
17640	(-0.47747119351222356, 6.103816654174744e-5, 21958.0)
17660	(-0.47738108146933445, 6.099444755275395e-5, 21958.0)
17680	(-0.47729103938418005, 6.09508097956992e-5, 21958.0)
17700	(-0.4772010671250077, 6.090725303052471e-5, 21958.0)
17720	(-0.4771111645604576, 6.0863777017660104e-5, 21958.0)
17740	(-0.4770213315595604, 6.0820381519038255e-5, 21958.0)
17760	(-0.4769315679917365, 6.077706629693096e-5, 21958.0)
17780	(-0.47684187372679426, 6.0733831114851416e-5, 21958.0)
17800	(-0.4767522486349283, 6.069067573710383e-5, 21958.0)
17820	(-0.4766626925867185, 6.0647599928821516e-5, 21958.0)
17840	(-0.47657320545312765, 6.060460345641866e-5, 21958.0)
17860	(-0.4764837871055009, 6.0561686086688586e-5, 21958.0)
17880	(-0.4763944374366027, 6.051883333650947e-5, 21958.0)
17900	(-0.47630515650011895, 6.0475936218638814

20340	(-0.4658910075354194, 5.578328277467611e-5, 21950.0)
20360	(-0.4658092906019101, 5.574868029095629e-5, 21950.0)
20380	(-0.46572762896429976, 5.571406037291329e-5, 21950.0)
20400	(-0.4656460231630921, 5.567906569385858e-5, 21950.0)
20420	(-0.4655644723703984, 5.564462992326079e-5, 21950.0)
20440	(-0.46548297649646053, 5.561024948422548e-5, 21950.0)
20460	(-0.4654015354517567, 5.557592423206029e-5, 21950.0)
20480	(-0.4653201491470002, 5.554165402276864e-5, 21950.0)
20500	(-0.4652388174931389, 5.5507438712595456e-5, 21950.0)
20520	(-0.465157540401354, 5.547327815863358e-5, 21950.0)
20540	(-0.4650763177830596, 5.54391722182179e-5, 21950.0)
20560	(-0.4649951495499017, 5.540512074926663e-5, 21950.0)
20580	(-0.4649140356137576, 5.537112361005419e-5, 21950.0)
20600	(-0.46483297588673467, 5.533718065970415e-5, 21950.0)
20620	(-0.46475197028117016, 5.530329175728002e-5, 21950.0)
20640	(-0.4646710187096296, 5.52694567629603e-5, 21950.0)
20660	(-0.46459012108490694, 5.523567553682605e-5, 219

23120	(-0.45502553792395006, 5.145010192114309e-5, 21958.0)
23140	(-0.45495072138111076, 5.1422045942756784e-5, 21958.0)
23160	(-0.45487594944356863, 5.139402955332774e-5, 21958.0)
23180	(-0.4548012220473861, 5.136605266069391e-5, 21958.0)
23200	(-0.4547265391287754, 5.1338115172771326e-5, 21958.0)
23220	(-0.4546519006240981, 5.1310216997821325e-5, 21958.0)
23240	(-0.45457730646986416, 5.1282358044603366e-5, 21958.0)
23260	(-0.4545027566027322, 5.125453822176457e-5, 21958.0)
23280	(-0.4544282509595085, 5.122675743856495e-5, 21958.0)
23300	(-0.4543537894771468, 5.1199015604381484e-5, 21958.0)
23320	(-0.4542793720927478, 5.117131262878449e-5, 21958.0)
23340	(-0.4542049987435585, 5.1143648422034065e-5, 21958.0)
23360	(-0.4541306693669721, 5.111602289412593e-5, 21958.0)
23380	(-0.45405638398203446, 5.108837989776497e-5, 21958.0)
23400	(-0.4539821425118187, 5.106078544233714e-5, 21958.0)
23420	(-0.4539079448877122, 5.103323382156578e-5, 21958.0)
23440	(-0.45383379098752613, 5.10057619005323

25900	(-0.44502813792228985, 4.7890957496824744e-5, 21957.0)
25920	(-0.44495897116184024, 4.786762934451145e-5, 21957.0)
25940	(-0.44488984137496274, 4.784433034128369e-5, 21957.0)
25960	(-0.4448207485146457, 4.782106042432175e-5, 21957.0)
25980	(-0.4447516925339787, 4.7797819531090454e-5, 21957.0)
26000	(-0.44468267338615186, 4.777460759953137e-5, 21957.0)
26020	(-0.4446136910244549, 4.7751424568063036e-5, 21957.0)
26040	(-0.44454474540227695, 4.772827037542736e-5, 21957.0)
26060	(-0.4444758375875911, 4.7704373373879485e-5, 21957.0)
26080	(-0.4444069665432012, 4.768119095598855e-5, 21957.0)
26100	(-0.4443381320989454, 4.765812293260209e-5, 21957.0)
26120	(-0.44426933420859765, 4.763508351123983e-5, 21957.0)
26140	(-0.444200572826029, 4.7612072632071186e-5, 21957.0)
26160	(-0.444131847905207, 4.758909023555122e-5, 21957.0)
26180	(-0.44406315940019564, 4.7566136262267e-5, 21957.0)
26200	(-0.4439945072651545, 4.754321065332211e-5, 21957.0)
26220	(-0.44392589145433914, 4.752031334949105e-

28680	(-0.4357496669282912, 4.490135290333721e-5, 21963.0)
28700	(-0.43568523113646623, 4.4881559291351004e-5, 21963.0)
28720	(-0.4356208266195966, 4.486178778923924e-5, 21963.0)
28740	(-0.4355564533420545, 4.484203835540032e-5, 21963.0)
28760	(-0.4354921112682797, 4.482231094810223e-5, 21963.0)
28780	(-0.4354278003627791, 4.48026055259079e-5, 21963.0)
28800	(-0.4353635205901274, 4.4782922047017865e-5, 21963.0)
28820	(-0.4352992719149654, 4.476326047058518e-5, 21963.0)
28840	(-0.4352350543020015, 4.474362075497514e-5, 21963.0)
28860	(-0.43517086771601005, 4.472400285938973e-5, 21963.0)
28880	(-0.43510671212183233, 4.470440674259126e-5, 21963.0)
28900	(-0.43504258748437574, 4.46848323637534e-5, 21963.0)
28920	(-0.4349784937686137, 4.466527968215179e-5, 21963.0)
28940	(-0.4349144309395858, 4.4645748656893244e-5, 21963.0)
28960	(-0.43485039896239724, 4.4626239247573503e-5, 21963.0)
28980	(-0.43478639781257583, 4.4606744194806365e-5, 21963.0)
29000	(-0.43472242748637263, 4.458724905783734e

In [55]:
β_orig = vcat(grpmat * collect(V.β[1:end-2]), collect(V.β)[end-1:end]);

In [56]:
for (v, β) in zip(variable_names_replicated[V.β .!= 0], V.β[V.β .!= 0])
    println("$v\t$β")
end

KEGG:00280: Valine, leucine and isoleucine degradation	ABAT	4-aminobutyrate aminotransferase	0.0008323544134017851
KEGG:00280: Valine, leucine and isoleucine degradation	AUH	AU RNA binding protein/enoyl-CoA hydratase	-0.0014593426888791475
KEGG:00280: Valine, leucine and isoleucine degradation	HMGCL	3-hydroxymethyl-3-methylglutaryl-CoA lyase	-0.0010615725170478399
KEGG:00280: Valine, leucine and isoleucine degradation	IVD	isovaleryl-CoA dehydrogenase	-0.000934855917150304
KEGG:00280: Valine, leucine and isoleucine degradation	MCCC1	methylcrotonoyl-CoA carboxylase 1 (alpha)	8.306605556755154e-5
KEGG:00280: Valine, leucine and isoleucine degradation	MCCC2	methylcrotonoyl-CoA carboxylase 2 (beta)	-0.000783494419844115
KEGG:00280: Valine, leucine and isoleucine degradation	MUT	methylmalonyl CoA mutase	0.0027548880164145434
KEGG:04727: GABAergic synapse	ABAT	4-aminobutyrate aminotransferase	0.0008232735329612727
KEGG:04727: GABAergic synapse	GABRA4	gamma-aminobutyric acid (GABA) A receptor,

KEGG:00650: Butanoate metabolism	ABAT	4-aminobutyrate aminotransferase	0.0008342785686320108
KEGG:00650: Butanoate metabolism	ALDH5A1	aldehyde dehydrogenase 5 family, member A1	0.002331490731821375
KEGG:00650: Butanoate metabolism	BDH1	3-hydroxybutyrate dehydrogenase, type 1	-0.00020734363851731562
KEGG:00650: Butanoate metabolism	HMGCL	3-hydroxymethyl-3-methylglutaryl-CoA lyase	-0.001064392202790947
GO:0001666: response to hypoxia	ABAT	4-aminobutyrate aminotransferase	0.0008334434534379149
GO:0001666: response to hypoxia	ADM	adrenomedullin	0.0022388125598973474
GO:0001666: response to hypoxia	AGER	advanced glycosylation end product-specific receptor	-0.00017426454237396926
GO:0001666: response to hypoxia	BNIP3	BCL2/adenovirus E1B 19kDa interacting protein 3	0.0029141508436547003
GO:0001666: response to hypoxia	CAV1	caveolin 1, caveolae protein, 22kDa	-0.0005354719528487099
GO:0001666: response to hypoxia	CCL2	chemokine (C-C motif) ligand 2	0.000817221336558294
GO:0001666: response to 

GO:0035094: response to nicotine	BCL2	B-cell CLL/lymphoma 2	-5.086636827417203e-6
GO:0035094: response to nicotine	CHRNB2	cholinergic receptor, nicotinic, beta 2 (neuronal)	-0.0007583278132899444
GO:0035094: response to nicotine	DRD2	dopamine receptor D2	-0.00023001114993561654
GO:0035094: response to nicotine	GPX1	glutathione peroxidase 1	0.00038673054565649693
GO:0035094: response to nicotine	HDAC2	histone deacetylase 2	-0.001188541499224591
GO:0035094: response to nicotine	HMOX1	heme oxygenase (decycling) 1	-0.0002107114039207383
GO:0035094: response to nicotine	PDX1	pancreatic and duodenal homeobox 1	0.0002469304872419899
GO:0035094: response to nicotine	STAR	steroidogenic acute regulatory protein	0.0008692233019067948
GO:0042135: neurotransmitter catabolic process	ABAT	4-aminobutyrate aminotransferase	0.0008384285830425876
GO:0042135: neurotransmitter catabolic process	ALDH5A1	aldehyde dehydrogenase 5 family, member A1	0.0023452776173856042
GO:0042493: response to drug	ABAT	4-amin

GO:0006355: regulation of transcription, DNA-templated	CEBPB	CCAAT/enhancer binding protein (C/EBP), beta	-0.00027454497597901374
GO:0006355: regulation of transcription, DNA-templated	CELSR2	cadherin, EGF LAG seven-pass G-type receptor 2	-0.001136098143241887
GO:0006355: regulation of transcription, DNA-templated	CHAF1B	chromatin assembly factor 1, subunit B (p60)	0.006292511066352087
GO:0006355: regulation of transcription, DNA-templated	CHD3	chromodomain helicase DNA binding protein 3	0.00011917125375918148
GO:0006355: regulation of transcription, DNA-templated	CLOCK	clock circadian regulator	0.0002003573870575256
GO:0006355: regulation of transcription, DNA-templated	DAXX	death-domain associated protein	0.0008926960114535768
GO:0006355: regulation of transcription, DNA-templated	DLX6	distal-less homeobox 6	-0.00018007477071055292
GO:0006355: regulation of transcription, DNA-templated	E2F1	E2F transcription factor 1	0.0021001730920734312
GO:0006355: regulation of transcription, DNA-

GO:0032367: intracellular cholesterol transport	ABCG1	ATP-binding cassette, sub-family G (WHITE), member 1	-0.0023414183670599836
GO:0032367: intracellular cholesterol transport	STAR	steroidogenic acute regulatory protein	0.0008955133587820496
GO:0032367: intracellular cholesterol transport	VPS4A	vacuolar protein sorting 4 homolog A (S. cerevisiae)	-0.0006857834708294201
GO:0033344: cholesterol efflux	ABCG1	ATP-binding cassette, sub-family G (WHITE), member 1	-0.0023351541238514502
GO:0033344: cholesterol efflux	CAV1	caveolin 1, caveolae protein, 22kDa	-0.0005351373609785365
GO:0033344: cholesterol efflux	SOAT1	sterol O-acyltransferase 1	-0.00040815997631377316
GO:0033700: phospholipid efflux	ABCG1	ATP-binding cassette, sub-family G (WHITE), member 1	-0.0023606226924747856
GO:0033993: response to lipid	ABCG1	ATP-binding cassette, sub-family G (WHITE), member 1	-0.0023480539884438473
GO:0033993: response to lipid	GATA2	GATA binding protein 2	-0.0004546861240076838
GO:0033993: response t

...

KEGG:04020: Calcium signaling pathway	PRKCA	protein kinase C, alpha	-5.844379105646568e-6
KEGG:04020: Calcium signaling pathway	PRKX	protein kinase, X-linked	0.00038599883480925154
KEGG:04020: Calcium signaling pathway	SPHK1	sphingosine kinase 1	0.001783148414863169
GO:0003081: regulation of systemic arterial blood pressure by renin-angiotensin	AGTR1	angiotensin II receptor, type 1	0.0003408727207204733
GO:0007266: Rho protein signal transduction	AGTR1	angiotensin II receptor, type 1	0.000347459360970462
GO:0007266: Rho protein signal transduction	CFL1	cofilin 1 (non-muscle)	-0.0011751921834503032
GO:0007266: Rho protein signal transduction	CNKSR1	connector enhancer of kinase suppressor of Ras 1	-0.001106153113808968
GO:0007266: Rho protein signal transduction	PAK1	p21 protein (Cdc42/Rac)-activated kinase 1	-0.0021878464033907876
GO:0007266: Rho protein signal transduction	PAK3	p21 protein (Cdc42/Rac)-activated kinase 3	-0.0012586585921412234
GO:0007266: Rho protein signal transduction

GO:0006915: apoptotic process	RAD21	RAD21 homolog (S. pombe)	-0.00010240308952538419
GO:0006915: apoptotic process	RAF1	Raf-1 proto-oncogene, serine/threonine kinase	0.0014982218551294762
GO:0006915: apoptotic process	S100A14	S100 calcium binding protein A14	-0.0021585857386697137
GO:0006915: apoptotic process	SIX1	SIX homeobox 1	-0.001869221072439771
GO:0006915: apoptotic process	STEAP3	STEAP family member 3, metalloreductase	0.002608239702649017
GO:0006915: apoptotic process	STK24	serine/threonine kinase 24	0.0026936240307968017
GO:0006915: apoptotic process	SULF1	sulfatase 1	-0.0008048270968838142
GO:0006915: apoptotic process	TFDP1	transcription factor Dp-1	0.0014437072999589982
GO:0006915: apoptotic process	TGFBR1	transforming growth factor, beta receptor 1	0.00033965305776263195
GO:0006915: apoptotic process	TNFAIP3	tumor necrosis factor, alpha-induced protein 3	0.0011160752793497313
GO:0006915: apoptotic process	TNFRSF10C	tumor necrosis factor receptor superfamily, member 10c, d

GO:0051260: protein homooligomerization	AKR1C1	aldo-keto reductase family 1, member C1	-0.0014765187659678443
GO:0051260: protein homooligomerization	AXIN1	axin 1	-0.0007229657882528065
GO:0051260: protein homooligomerization	BAX	BCL2-associated X protein	-0.0004218136086391652
GO:0051260: protein homooligomerization	CAV1	caveolin 1, caveolae protein, 22kDa	-0.0005317576552947027
GO:0051260: protein homooligomerization	CEP57	centrosomal protein 57kDa	-0.0005350088650690889
GO:0051260: protein homooligomerization	CPT1A	carnitine palmitoyltransferase 1A (liver)	0.001263172456800182
GO:0051260: protein homooligomerization	CRYAA	crystallin, alpha A	-0.0006664269036593953
GO:0051260: protein homooligomerization	CRYAB	crystallin, alpha B	0.0009705595613955781
GO:0051260: protein homooligomerization	ECT2	epithelial cell transforming 2	0.0010495059974709275
GO:0051260: protein homooligomerization	FAS	Fas cell surface death receptor	-3.284126954160684e-5
GO:0051260: protein homooligomerization	

Excessive output truncated after 524306 bytes.

KEGG:05166: HTLV-I infection	WNT7A	wingless-type MMTV integration site family, member 7A	1.9579561150038085e-5
KEGG:05166: HTLV-I infection	XBP1	X-box binding protein 1	0.000266800147805487
KEGG:05169: Epstein-Barr virus infection	AKT1	v-akt murine thymoma viral oncogene homolog 1	0.0007262566190756878
KEGG:05169: Epstein-Barr virus infection	AKT2	v-akt murine thymoma viral oncogene homolog 2	-0.0006235672077896341
KEGG:05169: Epstein-Barr virus infection	BCL2	B-cell CLL/lymphoma 2	-4.88293798505856e-6
KEGG:05169: Epstein-Barr virus infection	CCNA1	cyclin A1	-0.0017501487999748554
KEGG:05169: Epstein-Barr virus infection	CDK1	cyclin-dependent kinase 1	0.0003069529137300432
KEGG:05169: Epstein-Barr virus infection	CSNK2B	casein kinase 2, beta polypeptide	0.001519276904370727
KEGG:05169: Epstein-Barr virus infection	GSK3B	glycogen synthase kinase 3 beta	0.0015411902495898234
KEGG:05169: Epstein-Barr virus infection	HDAC2	histone deacetylase 2	-0.0012038988901816757
KEGG:05169: Epstein-Ba

In [57]:
for (v, β) in zip(variable_names[β_orig .!= 0], β_orig[β_orig .!= 0])
    println("$v\t$β")
end

RFC2	0.01782837035434921
PAX8	0.03684757874519627
THRA	0.02806710700734747
EPHB3	0.019420774902517743
CFL1	-0.030460550648403983
YY1	0.013402782876598919
ZPR1	-0.004170459413053395
RHOA	0.04322099220670632
GUK1	0.052841615328390566
DSP	-0.0008924333259685146
RAD21	-0.0016314685844209603
SF3B2	0.004117778492081898
NDRG1	-0.006543628992920168
CD63	-0.01588590376579995
DNAJB1	-0.002074023570791528
XBP1	0.02116123361651641
SPTBN1	0.03684346496032823
HMGB1	-0.030756596050312536
GPX1	0.02123400737801508
NUMA1	0.02134145223823748
ARL6IP5	0.0010924161799196037
STMN1	0.007247922509085317
ODC1	0.009846896972186726
DAZAP2	-0.0007855012627521646
TMBIM6	-0.0022818769304895877
GSTP1	0.008155197247241446
ZNF207	-0.002720556617926771
DHCR24	-0.03582951324451189
DEK	0.017525131494012178
HIF1A	-0.049801244025156705
SNX17	0.011478189129344105
CD9	-0.038892803436755025
JUP	-0.038198768007394066
TGM2	-0.0017271186611303273
MMP2	-0.025981932630592185
THBS1	-0.0844641572375033
TUFM	-0.006526015058534713
POLD

SLC4A7	0.010467453451719635
FOXH1	-0.024641509894376825
ADAM2	0.006458059209497759
PAX3	-0.05906825353319958
KIF25	-0.009609396255554067
SERPINH1	-0.00756363647970954
PPP2R3A	-0.0006220842487928169
RASSF8	0.0033201926705128007
RBPJ	0.05165924746472585
KRT33B	0.0018854914919091375
POU1F1	0.0019684636080322788
MUC1	0.01534045317873626
AVP	-0.03623974017775201
INSR	-0.07200559687285481
NAALADL1	-0.0010165593282607673
IL12B	0.09369902304859444
GATA2	-0.019232401633833177
MEF2C	0.09777138637302661
CLEC4M	-0.024936805204337182
RFC1	0.023320146009614226
CDC14B	0.0001570744684361422
HMGA2	0.07233033972660002
LAIR1	0.0012946246181076426
CCL7	-0.05525290613251587
ZNF442	0.0021217095248059516
NDEL1	-0.014938650791860793
ANP32E	0.0008414262211640946
DDX17	0.01062450490057107
CHIT1	0.0017042028202982383
PSG4	-0.0010328661238374213
TTN	0.03969133562208976
MAS1	0.005871102423210322
DRD4	-0.07577515006143415
HOXB1	0.0481776052277343
TH	-0.02298980468540806
PGR	0.005606482706949607
CSH1	-0.001029440511

DIABLO	-0.005344035959869675
ATHL1	-0.0009901807536505773
KLF2	-0.023289426706930974
GRHL2	0.025092132700132403
TTYH1	0.009566840787928558
COA7	0.0005966881548589196
TTC33	-0.0026210948750966644
SULT4A1	-0.005814538701167442
EGFL6	0.0008221974015097792
C7orf63	0.0012350401404626573
LAMP5	-0.0035792461217451195
LRFN4	0.0013939386408319417
GCNT3	-0.02550658691485919
CEP72	-0.01642567049933524
BORA	0.011964177974982488
ROBO3	0.0047998153602208495
SOX18	0.029500021112874815
KIF16B	0.03523818751096763
PIEZO2	0.0020898293231342577
CEP83	-0.0007677935602652968
PTPLA	0.0025548743142632946
TBX3	0.05010579689726048
FZD3	0.009331988846620861
BBS7	-0.020445120694412334
TFCP2L1	0.00485936774826609
HEY2	0.13883585381515867
VTCN1	-0.011060391877614668
GIMAP6	-0.0007352876517308439
MTL5	-0.0025622556198957327
ECT2	0.024320114192369516
ARL15	-0.004536728216233623
NARF	0.0014994215563336728
FAM198B	9.679876491163129e-5
LHX6	0.009407866917839713
SPAG4	0.0006137799805040552
ASPM	0.0002548203396439097
GLRX